# Gweizy Model Training Notebook

Train all gas prediction models for Gweizy.

## Instructions:
1. Upload your `gas_data.db` file (from `backend/gas_data.db`)
2. Run all cells
3. Download the trained models zip file
4. Extract to `backend/models/saved_models/` and push to GitHub

In [ ]:
# Install dependencies
!pip install -q scikit-learn pandas numpy joblib lightgbm xgboost matplotlib seaborn optuna

In [ ]:
# Upload your gas_data.db file
from google.colab import files
import os

print("Upload your gas_data.db file from backend/gas_data.db")
uploaded = files.upload()

if 'gas_data.db' in uploaded:
    print(f"\n✅ Uploaded gas_data.db ({len(uploaded['gas_data.db']) / 1024 / 1024:.1f} MB)")
else:
    print("❌ Please upload gas_data.db")

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Load data from database
conn = sqlite3.connect('gas_data.db')
df = pd.read_sql("""
    SELECT timestamp, current_gas as gas, base_fee, priority_fee, 
           block_number, gas_used, gas_limit, utilization
    FROM gas_prices ORDER BY timestamp ASC
""", conn)
conn.close()

df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.set_index('timestamp').sort_index()

print(f"Total records: {len(df):,}")
print(f"Date range: {df.index.min()} to {df.index.max()}")

# === IMPROVED: Resample to 30-second intervals (was 1-min, losing too much data) ===
print("\nResampling to 30-second intervals (preserves more data)...")
df = df.resample('30s').mean().dropna(subset=['gas'])
print(f"After resample: {len(df):,} records")

# Find segments (gap > 30 min = new segment)
df['time_diff'] = df.index.to_series().diff()
df['segment'] = (df['time_diff'] > pd.Timedelta(minutes=30)).cumsum()

segment_sizes = df.groupby('segment').size()
print(f"\nSegments found: {len(segment_sizes)}")
print(f"Segment sizes: {segment_sizes.sort_values(ascending=False).head(10).tolist()}")

# === IMPROVED: Lower threshold from 120 to 30 minutes (keeps more segments) ===
MIN_SEGMENT_SIZE = 60  # 30 minutes at 30-sec intervals = 60 records
good_segments = segment_sizes[segment_sizes >= MIN_SEGMENT_SIZE].index.tolist()
df = df[df['segment'].isin(good_segments)]
print(f"\nKeeping {len(good_segments)} segments with >= 30 minutes of data")
print(f"Total usable records: {len(df):,}")

# === DATA SUFFICIENCY CHECK ===
MIN_REQUIRED_SAMPLES = 10000
if len(df) < MIN_REQUIRED_SAMPLES:
    print(f"\n⚠️  WARNING: Only {len(df):,} samples. Recommend at least {MIN_REQUIRED_SAMPLES:,}")
    print("   Models may underperform. Consider collecting more data.")
else:
    print(f"\n✓ Data sufficiency check passed: {len(df):,} samples")

RECORDS_PER_HOUR = 120  # 30-sec intervals = 120 records per hour

In [ ]:
# Fetch ETH Price Data - IMPROVED with Binance (1-minute data)
import requests

print("="*60)
print("FETCHING EXTERNAL DATA")
print("="*60)

def fetch_eth_price_binance(start_date, end_date):
    """Fetch ETH price from Binance API (1-minute candles, much better than CoinGecko hourly)"""
    try:
        start_ts = int(start_date.timestamp() * 1000)
        end_ts = int(end_date.timestamp() * 1000)
        
        all_prices = []
        current_ts = start_ts
        
        print(f"Fetching ETH prices from Binance (1-min candles)...")
        
        while current_ts < end_ts:
            url = "https://api.binance.com/api/v3/klines"
            params = {
                'symbol': 'ETHUSDT',
                'interval': '1m',
                'startTime': current_ts,
                'endTime': min(current_ts + 1000 * 60 * 1000, end_ts),  # Max 1000 candles
                'limit': 1000
            }
            
            response = requests.get(url, params=params, timeout=30)
            
            if response.status_code == 200:
                data = response.json()
                if not data:
                    break
                    
                for candle in data:
                    all_prices.append({
                        'timestamp': pd.to_datetime(candle[0], unit='ms'),
                        'eth_price': float(candle[4]),  # Close price
                        'eth_volume': float(candle[5]),  # Volume
                        'eth_high': float(candle[2]),
                        'eth_low': float(candle[3])
                    })
                
                current_ts = data[-1][0] + 60000  # Next minute
                
                if len(all_prices) % 5000 == 0:
                    print(f"  Fetched {len(all_prices):,} candles...")
            else:
                print(f"  Binance API error: {response.status_code}")
                break
        
        if all_prices:
            eth_df = pd.DataFrame(all_prices)
            eth_df = eth_df.set_index('timestamp')
            print(f"  Total: {len(eth_df):,} 1-minute ETH candles")
            return eth_df
        return None
        
    except Exception as e:
        print(f"  Failed to fetch from Binance: {e}")
        return None

def fetch_eth_price_coingecko(start_date, end_date):
    """Fallback: CoinGecko API (hourly data)"""
    try:
        start_ts = int(start_date.timestamp())
        end_ts = int(end_date.timestamp())
        
        url = "https://api.coingecko.com/api/v3/coins/ethereum/market_chart/range"
        params = {'vs_currency': 'usd', 'from': start_ts, 'to': end_ts}
        
        print(f"Fallback: Fetching from CoinGecko (hourly)...")
        response = requests.get(url, params=params, timeout=30)
        
        if response.status_code == 200:
            data = response.json()
            prices = data.get('prices', [])
            
            eth_df = pd.DataFrame(prices, columns=['timestamp', 'eth_price'])
            eth_df['timestamp'] = pd.to_datetime(eth_df['timestamp'], unit='ms')
            eth_df = eth_df.set_index('timestamp')
            eth_df['eth_volume'] = np.nan
            eth_df['eth_high'] = eth_df['eth_price']
            eth_df['eth_low'] = eth_df['eth_price']
            
            print(f"  Fetched {len(eth_df)} hourly ETH prices")
            return eth_df
        return None
    except Exception as e:
        print(f"  CoinGecko failed: {e}")
        return None

# Try Binance first, fallback to CoinGecko
eth_data = fetch_eth_price_binance(df.index.min(), df.index.max())
if eth_data is None or len(eth_data) < 100:
    eth_data = fetch_eth_price_coingecko(df.index.min(), df.index.max())

has_eth_data = False
if eth_data is not None and len(eth_data) > 0:
    # Resample to 30-second intervals
    eth_data = eth_data.resample('30s').ffill()
    
    # Merge with gas data
    df = df.join(eth_data, how='left')
    df['eth_price'] = df['eth_price'].ffill().bfill()
    
    # Fill other ETH columns
    for col in ['eth_volume', 'eth_high', 'eth_low']:
        if col in df.columns:
            df[col] = df[col].ffill().bfill()
    
    eth_coverage = df['eth_price'].notna().mean()
    print(f"  ETH price coverage: {eth_coverage:.1%}")
    
    if eth_coverage > 0.5:
        has_eth_data = True
        print("  ✓ ETH price data integrated (1-min resolution)")
else:
    print("  ⚠️ No ETH price data available")
    df['eth_price'] = np.nan
    df['eth_volume'] = np.nan
    df['eth_high'] = np.nan
    df['eth_low'] = np.nan

HAS_ETH_PRICE = has_eth_data

In [ ]:
# Feature Engineering - SIMPLIFIED v3 + SPIKE-ADJUSTED TARGETS# Focus: 15-20 high-value features to prevent overfitting# NEW: Option for log-transformed or winsorized targetsprint("Engineering SIMPLIFIED feature set (15-20 features)...")# === CONFIGURATION ===TARGET_TRANSFORM = "log"  # Options: "none", "log", "winsorize"WINSORIZE_PERCENTILE = 0.95  # For winsorize: cap at this percentiledef engineer_features_for_segment(seg_df, has_eth=False, horizon='all'):    """Engineer focused feature set - quality over quantity"""    df = seg_df.copy()    rph = 120  # records per hour (30-sec intervals)        # === TIME FEATURES (3 features) ===    df['hour'] = df.index.hour    hour_of_day = df.index.hour + df.index.minute / 60    df['hour_sin'] = np.sin(2 * np.pi * hour_of_day / 24)    df['hour_cos'] = np.cos(2 * np.pi * hour_of_day / 24)        # === ETH FEATURES (2 features) ===    if has_eth and 'eth_price' in df.columns and df['eth_price'].notna().any():        df['eth_log'] = np.log1p(df['eth_price'])        eth_mean = df['eth_price'].rolling(4*rph, min_periods=rph).mean()        eth_std = df['eth_price'].rolling(4*rph, min_periods=rph).std()        df['eth_zscore_4h'] = np.where(eth_std > 0.01, (df['eth_price'] - eth_mean) / eth_std, 0)        df['gas_eth_corr_1h'] = df['gas'].rolling(rph, min_periods=rph//2).corr(df['eth_price']).fillna(0)        # === NETWORK UTILIZATION (2 features) ===    if 'utilization' in df.columns:        df['util_mean_1h'] = df['utilization'].rolling(rph, min_periods=rph//2).mean()        df['util_mean_2h'] = df['utilization'].rolling(2*rph, min_periods=rph).mean()        # === GAS LAG FEATURES (5 features) ===    df['gas_lag_5min'] = df['gas'].shift(10)    df['gas_lag_15min'] = df['gas'].shift(30)    df['gas_lag_30min'] = df['gas'].shift(60)    df['gas_lag_1h'] = df['gas'].shift(rph)    df['gas_lag_4h'] = df['gas'].shift(4*rph)        # === ROLLING STATS (6 features) ===    df['gas_mean_1h'] = df['gas'].rolling(rph, min_periods=rph//2).mean()    df['gas_std_1h'] = df['gas'].rolling(rph, min_periods=rph//2).std()    df['gas_cv_1h'] = np.where(df['gas_mean_1h'] > 0.01,                                 df['gas_std_1h'] / df['gas_mean_1h'], 0)    df['gas_mean_2h'] = df['gas'].rolling(2*rph, min_periods=rph).mean()    df['gas_mean_4h'] = df['gas'].rolling(4*rph, min_periods=rph).mean()        # === MOMENTUM (3 features) ===    df['momentum_1h'] = df['gas'] - df['gas'].shift(rph)    shift_2h = df['gas'].shift(2*rph)    df['momentum_pct_2h'] = np.where(shift_2h > 0.01, (df['gas'] - shift_2h) / shift_2h, 0)    df['trend_1h_4h'] = np.where(df['gas_mean_4h'] > 0.01, df['gas_mean_1h'] / df['gas_mean_4h'], 1.0)        # === Z-SCORE AND REGIME (3 features) ===    df['gas_zscore_1h'] = np.where(df['gas_std_1h'] > 0.001,         (df['gas'] - df['gas_mean_1h']) / df['gas_std_1h'], 0)    df['is_spike'] = (df['gas'] > df['gas_mean_1h'] + 2 * df['gas_std_1h']).astype(int)    df['is_high_gas'] = (df['gas'] > df['gas'].rolling(4*rph, min_periods=rph).quantile(0.9)).astype(int)        return df# Process each segmentprint("\nProcessing segments...")segments = df['segment'].unique()processed_segments = []for seg_id in segments:    seg_df = df[df['segment'] == seg_id].copy()    processed = engineer_features_for_segment(seg_df, has_eth=has_eth_data, horizon='all')    processed_segments.append(processed)df_features = pd.concat(processed_segments, axis=0)print(f"After feature engineering: {len(df_features):,} records")# Create targetsprint("\nCreating prediction targets...")def create_targets_for_segment(seg_df, transform="none", winsorize_pct=0.95):    """Create target variables with optional transformation"""    df = seg_df.copy()    rph = 120        # Raw future prices    raw_1h = df['gas'].shift(-rph)    raw_4h = df['gas'].shift(-4*rph)    raw_24h = df['gas'].shift(-24*rph)        # Apply transformation    if transform == "log":        # Log transform - better for multiplicative changes        df['target_1h'] = np.log1p(raw_1h)        df['target_4h'] = np.log1p(raw_4h)        df['target_24h'] = np.log1p(raw_24h)        # Also store raw for evaluation        df['target_1h_raw'] = raw_1h        df['target_4h_raw'] = raw_4h        df['target_24h_raw'] = raw_24h    elif transform == "winsorize":        # Winsorize - cap extreme values        cap_1h = raw_1h.quantile(winsorize_pct)        cap_4h = raw_4h.quantile(winsorize_pct)        cap_24h = raw_24h.quantile(winsorize_pct) if raw_24h.notna().sum() > 100 else cap_4h        df['target_1h'] = raw_1h.clip(upper=cap_1h)        df['target_4h'] = raw_4h.clip(upper=cap_4h)        df['target_24h'] = raw_24h.clip(upper=cap_24h)        df['target_1h_raw'] = raw_1h        df['target_4h_raw'] = raw_4h        df['target_24h_raw'] = raw_24h        print(f"  Winsorized caps: 1h={cap_1h:.2f}, 4h={cap_4h:.2f}")    else:        # No transform        df['target_1h'] = raw_1h        df['target_4h'] = raw_4h        df['target_24h'] = raw_24h        # Direction classification (always on raw)    threshold = 0.02    for horizon in ['1h', '4h']:        raw_target = raw_1h if horizon == '1h' else raw_4h        pct_change = np.where(df['gas'] > 0.001,             (raw_target - df['gas']) / df['gas'], 0)        df[f'direction_class_{horizon}'] = pd.cut(            pct_change,            bins=[-float('inf'), -threshold, threshold, float('inf')],            labels=['down', 'stable', 'up']        )        return dfprint(f"Target transform: {TARGET_TRANSFORM}")processed_with_targets = []for seg_id in df_features['segment'].unique():    seg_df = df_features[df_features['segment'] == seg_id].copy()    processed = create_targets_for_segment(seg_df, transform=TARGET_TRANSFORM, winsorize_pct=WINSORIZE_PERCENTILE)    processed_with_targets.append(processed)df_features = pd.concat(processed_with_targets, axis=0)# Store transform info for later useTARGET_TRANSFORM_USED = TARGET_TRANSFORM# === CLEAN INF/NAN VALUES ===print("\nCleaning inf/nan values...")numeric_cols = df_features.select_dtypes(include=[np.number]).columnsfor col in numeric_cols:    df_features[col] = df_features[col].replace([np.inf, -np.inf], np.nan)    if df_features[col].notna().sum() > 0:        q_low = df_features[col].quantile(0.001)        q_high = df_features[col].quantile(0.999)        df_features[col] = df_features[col].clip(q_low, q_high)df_features = df_features.ffill().bfill()for col in numeric_cols:    if df_features[col].isna().any():        median_val = df_features[col].median()        if pd.isna(median_val):            median_val = 0        df_features[col] = df_features[col].fillna(median_val)inf_count = np.isinf(df_features.select_dtypes(include=[np.number])).sum().sum()nan_count = df_features.select_dtypes(include=[np.number]).isna().sum().sum()print(f"  After cleaning: {inf_count} inf, {nan_count} nan values")# === DEFINE FOCUSED FEATURE SET ===# === EXTERNAL DATA INTEGRATION HOOKS ===# These features will be used if the corresponding data is availabledef add_external_features(df):    """    Add external data features when available.    Currently supports: blob_gas, l1_congestion, day_of_week    """    external_added = []    # === BLOB GAS (EIP-4844) ===    if 'blob_gas_price' in df.columns:        df['blob_gas_log'] = np.log1p(df['blob_gas_price'])        df['blob_vs_gas_ratio'] = df['blob_gas_price'] / (df['gas'] + 1e-8)        external_added.extend(['blob_gas_log', 'blob_vs_gas_ratio'])    # === L1 CONGESTION ===    if 'l1_gas_price' in df.columns:        df['l1_gas_log'] = np.log1p(df['l1_gas_price'])        df['l2_l1_ratio'] = df['gas'] / (df['l1_gas_price'] + 1e-8)        external_added.extend(['l1_gas_log', 'l2_l1_ratio'])    # === DAY OF WEEK (always available from timestamp) ===    if hasattr(df.index, 'dayofweek'):        df['day_of_week'] = df.index.dayofweek        df['is_weekend'] = (df.index.dayofweek >= 5).astype(int)        # Cyclical encoding        df['dow_sin'] = np.sin(2 * np.pi * df.index.dayofweek / 7)        df['dow_cos'] = np.cos(2 * np.pi * df.index.dayofweek / 7)        external_added.extend(['day_of_week', 'is_weekend', 'dow_sin', 'dow_cos'])    # === HOLIDAY INDICATOR (placeholder) ===    # If holiday data is provided in a 'is_holiday' column    if 'is_holiday' in df.columns:        external_added.append('is_holiday')    return df, external_added# Apply external features to all segmentsprint("\nAdding external data features...")if 'segment' not in df_features.columns:    print('  Note: No segment column, treating as single segment')    df_features['segment'] = 0external_features_added = []processed_with_external = []for seg_id in df_features['segment'].unique():    seg_df = df_features[df_features['segment'] == seg_id].copy()    seg_df, added = add_external_features(seg_df)    processed_with_external.append(seg_df)    if not external_features_added:        external_features_added = addeddf_features = pd.concat(processed_with_external, axis=0)if external_features_added:    print(f"  Added external features: {external_features_added}")else:    print("  No external data columns found, using standard features only")# === DEFINE FOCUSED FEATURE SET ===CORE_FEATURES = [    'hour', 'hour_sin', 'hour_cos',    'eth_log', 'eth_zscore_4h', 'gas_eth_corr_1h',    'util_mean_1h', 'util_mean_2h',    'gas_lag_5min', 'gas_lag_15min', 'gas_lag_30min', 'gas_lag_1h', 'gas_lag_4h',    'gas_mean_1h', 'gas_std_1h', 'gas_cv_1h', 'gas_mean_2h', 'gas_mean_4h',    'momentum_1h', 'momentum_pct_2h', 'trend_1h_4h',    'gas_zscore_1h', 'is_spike', 'is_high_gas']# Add external features to core if availableEXTENDED_FEATURES = CORE_FEATURES + [    'day_of_week', 'is_weekend', 'dow_sin', 'dow_cos',  # Day of week    'blob_gas_log', 'blob_vs_gas_ratio',  # Blob gas    'l1_gas_log', 'l2_l1_ratio',  # L1 congestion    'is_holiday'  # Holiday]# Use extended features if available, otherwise coreavailable_features = [f for f in EXTENDED_FEATURES if f in df_features.columns]if len(available_features) == len([f for f in CORE_FEATURES if f in df_features.columns]):    print("  Using core features only")else:    print(f"  Using extended features ({len(available_features)} total)")features_1h = available_featuresfeatures_4h = available_features  features_24h = available_featuresprint(f"\n✓ Focused feature set: {len(available_features)} features")print(f"  Features: {', '.join(available_features)}")

In [ ]:
# Prepare training data - with AUTO-ADAPT to distribution shift
# Prepare training data - with AUTO-ADAPT to distribution shiftfrom sklearn.preprocessing import RobustScaler# === SAFETY CHECK ===required_vars = ['df_features', 'features_1h', 'features_4h', 'features_24h']missing = [v for v in required_vars if v not in dir()]if missing:    raise RuntimeError(f"Missing variables from Cell 5: {missing}. Please run Cell 5 (Feature Engineering) first!")print(f"✓ df_features loaded: {len(df_features):,} rows")print(f"✓ Features: 1h={len(features_1h)}, 4h={len(features_4h)}, 24h={len(features_24h)}")from sklearn.preprocessing import RobustScaler
from scipy import stats

# === CONFIGURATION ===
USE_ROLLING_WINDOW = False  # Set True to use only recent data (AUTO-ENABLED if shift detected)
ROLLING_WINDOW_DAYS = 7     # Days of data to use if rolling window enabled
HOLDOUT_HOURS = 48          # Hours to reserve for holdout
AUTO_ADAPT_ON_SHIFT = True  # Automatically adapt when distribution shift detected

# === VOLATILITY ADAPTATION SETTINGS ===
# Shorter windows for more aggressive adaptation during volatile periods
SEVERE_SHIFT_WINDOW_DAYS = 1.5    # Very aggressive - 36 hours
HIGH_SHIFT_WINDOW_DAYS = 2        # Aggressive - 2 days  
MODERATE_SHIFT_WINDOW_DAYS = 3    # Moderate - 3 days
MILD_SHIFT_WINDOW_DAYS = 5        # Mild - 5 days

# Only keep numeric columns
numeric_features_1h = df_features[features_1h].select_dtypes(include=[np.number]).columns.tolist()
numeric_features_4h = df_features[features_4h].select_dtypes(include=[np.number]).columns.tolist()
numeric_features_24h = df_features[features_24h].select_dtypes(include=[np.number]).columns.tolist()

print(f"Numeric features: 1h={len(numeric_features_1h)}, 4h={len(numeric_features_4h)}, 24h={len(numeric_features_24h)}")

# Drop rows only where TARGET columns are NaN
target_cols = ['target_1h', 'target_4h']
df_clean = df_features.dropna(subset=target_cols)
print(f"Clean samples (with valid 1h/4h targets): {len(df_clean):,}")

valid_24h = df_features['target_24h'].notna().sum()
print(f"Samples with valid 24h target: {valid_24h:,}")

# === OUT-OF-TIME HOLDOUT (do this FIRST to detect shift) ===
rph = 120  # records per hour
holdout_size = HOLDOUT_HOURS * rph

if len(df_clean) > holdout_size + 5000:
    df_train_val_initial = df_clean.iloc[:-holdout_size]
    df_holdout = df_clean.iloc[-holdout_size:]
    print(f"\n✓ Out-of-time holdout: {len(df_holdout):,} samples (last {HOLDOUT_HOURS}h)")
    HAS_HOLDOUT = True
else:
    df_train_val_initial = df_clean
    df_holdout = None
    print(f"\n⚠️ Not enough data for holdout, using all for training")
    HAS_HOLDOUT = False

# === DISTRIBUTION SHIFT DETECTION ===
def detect_distribution_shift(train_data, holdout_data, name=""):
    """Detect distribution shift between train and holdout"""
    results = {'name': name, 'warnings': [], 'passed': True, 'shift_magnitude': 0}
    
    train_mean, train_std = train_data.mean(), train_data.std()
    holdout_mean = holdout_data.mean()
    mean_shift = abs(holdout_mean - train_mean) / (train_std + 1e-8)
    results['mean_shift_std'] = mean_shift
    
    if mean_shift > 1.0:
        results['warnings'].append(f"Large mean shift: {mean_shift:.2f} std devs")
        results['passed'] = False
        results['shift_magnitude'] = max(results['shift_magnitude'], mean_shift)
    elif mean_shift > 0.5:
        results['warnings'].append(f"Moderate mean shift: {mean_shift:.2f} std devs")
    
    var_ratio = holdout_data.var() / (train_data.var() + 1e-8)
    results['var_ratio'] = var_ratio
    
    if var_ratio > 4 or var_ratio < 0.25:
        results['warnings'].append(f"Large variance change: {var_ratio:.2f}x")
        results['passed'] = False
        results['shift_magnitude'] = max(results['shift_magnitude'], abs(np.log(var_ratio)))
    
    ks_stat, ks_pval = stats.ks_2samp(train_data.values[:5000], holdout_data.values[:5000])
    results['ks_statistic'] = ks_stat
    results['ks_pvalue'] = ks_pval
    
    if ks_pval < 0.001 and ks_stat > 0.3:
        results['warnings'].append(f"KS test: distributions differ significantly")
        results['passed'] = False
        results['shift_magnitude'] = max(results['shift_magnitude'], ks_stat * 3)
    
    train_spikes = (train_data > train_data.quantile(0.95)).mean()
    holdout_spikes = (holdout_data > train_data.quantile(0.95)).mean()
    spike_ratio = holdout_spikes / (train_spikes + 1e-8)
    results['spike_ratio'] = spike_ratio
    
    if spike_ratio > 3:
        results['warnings'].append(f"Spike frequency {spike_ratio:.1f}x higher in holdout")
        results['passed'] = False
        results['shift_magnitude'] = max(results['shift_magnitude'], spike_ratio / 2)
    
    return results

DISTRIBUTION_SHIFT_DETECTED = False
SHIFT_MAGNITUDE = 0

if HAS_HOLDOUT:
    print(f"\n{'='*60}")
    print("DISTRIBUTION SHIFT DETECTION")
    print(f"{'='*60}")
    
    for horizon in ['1h', '4h']:
        target_col = f'target_{horizon}'
        train_targets = df_train_val_initial[target_col].dropna()
        holdout_targets = df_holdout[target_col].dropna()
        
        shift_result = detect_distribution_shift(train_targets, holdout_targets, f"{horizon} target")
        
        status = "✓ OK" if shift_result['passed'] else "⚠️ SHIFT DETECTED"
        print(f"\n{horizon}: {status}")
        print(f"  Train:   mean={train_targets.mean():.4f}, std={train_targets.std():.4f}")
        print(f"  Holdout: mean={holdout_targets.mean():.4f}, std={holdout_targets.std():.4f}")
        print(f"  Mean shift: {shift_result['mean_shift_std']:.2f} std, Var ratio: {shift_result['var_ratio']:.2f}x")
        
        if shift_result['warnings']:
            for w in shift_result['warnings']:
                print(f"  ⚠️ {w}")
            DISTRIBUTION_SHIFT_DETECTED = True
            SHIFT_MAGNITUDE = max(SHIFT_MAGNITUDE, shift_result['shift_magnitude'])

# === AUTO-ADAPT TO DISTRIBUTION SHIFT (SHORTER WINDOWS) ===
if DISTRIBUTION_SHIFT_DETECTED and AUTO_ADAPT_ON_SHIFT:
    print(f"\n{'='*60}")
    print("AUTO-ADAPTING TO DISTRIBUTION SHIFT")
    print(f"{'='*60}")
    
    # More aggressive adaptive window based on shift magnitude
    if SHIFT_MAGNITUDE > 4:
        adaptive_days = SEVERE_SHIFT_WINDOW_DAYS  # 1.5 days (very aggressive)
        volatility_level = "SEVERE"
    elif SHIFT_MAGNITUDE > 2:
        adaptive_days = HIGH_SHIFT_WINDOW_DAYS    # 2 days (aggressive)
        volatility_level = "HIGH"
    elif SHIFT_MAGNITUDE > 1:
        adaptive_days = MODERATE_SHIFT_WINDOW_DAYS  # 3 days
        volatility_level = "MODERATE"
    else:
        adaptive_days = MILD_SHIFT_WINDOW_DAYS    # 5 days
        volatility_level = "MILD"
    
    window_samples = int(adaptive_days * 24 * rph)
    
    if len(df_train_val_initial) > window_samples:
        df_train_val = df_train_val_initial.iloc[-window_samples:]
        print(f"✓ Auto-enabled rolling window: {adaptive_days} days ({len(df_train_val):,} samples)")
        print(f"  Volatility: {volatility_level} (shift magnitude: {SHIFT_MAGNITUDE:.2f})")
        print(f"  Using most recent {adaptive_days} days for training")
        USE_ROLLING_WINDOW = True
        ROLLING_WINDOW_DAYS = adaptive_days
    else:
        df_train_val = df_train_val_initial
        print(f"⚠️ Not enough data for adaptive window, using all training data")
elif USE_ROLLING_WINDOW:
    # Manual rolling window
    window_samples = int(ROLLING_WINDOW_DAYS * 24 * rph)
    if len(df_train_val_initial) > window_samples:
        df_train_val = df_train_val_initial.iloc[-window_samples:]
        print(f"\n✓ Rolling window: Using last {ROLLING_WINDOW_DAYS} days ({len(df_train_val):,} samples)")
    else:
        df_train_val = df_train_val_initial
else:
    df_train_val = df_train_val_initial

print(f"\nFinal training set: {len(df_train_val):,} samples")

# Final safety check
for col in df_train_val.select_dtypes(include=[np.float64, np.float32, float]).columns:
    df_train_val[col] = df_train_val[col].replace([np.inf, -np.inf], np.nan)
    if df_train_val[col].isna().any():
        df_train_val[col] = df_train_val[col].fillna(df_train_val[col].median())

float_cols = df_train_val.select_dtypes(include=[np.float64, np.float32, float]).columns
has_inf = any(np.isinf(df_train_val[col]).any() for col in float_cols)
has_nan = any(np.isnan(df_train_val[col]).any() for col in float_cols)
assert not has_inf, "Data still contains inf!"
assert not has_nan, "Data still contains nan!"
print("✓ Data validated: no inf/nan values")

# Prepare feature matrices
X_1h = df_train_val[numeric_features_1h]
X_4h = df_train_val[numeric_features_4h]
X_24h = df_train_val[numeric_features_24h]

y_1h = df_train_val['target_1h']
y_4h = df_train_val['target_4h']
y_24h = df_train_val['target_24h']

y_dir_1h = df_train_val['direction_class_1h']
y_dir_4h = df_train_val['direction_class_4h']

current_gas = df_train_val['gas']

# === BASELINE MODELS (on both train AND holdout) ===
print(f"\n{'='*60}")
print("BASELINE COMPARISONS")
print(f"{'='*60}")

naive_mae_1h = np.mean(np.abs(y_1h.values - current_gas.values))
naive_mae_4h = np.mean(np.abs(y_4h.values - current_gas.values))

mean_pred = np.full_like(y_1h.values, y_1h.mean())
mean_mae_1h = np.mean(np.abs(y_1h.values - mean_pred))
mean_mae_4h = np.mean(np.abs(y_4h.values - mean_pred))

print(f"\nTRAINING SET Baseline MAEs:")
print(f"  Naive (current price):     MAE_1h={naive_mae_1h:.6f}, MAE_4h={naive_mae_4h:.6f}")
print(f"  Mean (historical average): MAE_1h={mean_mae_1h:.6f}, MAE_4h={mean_mae_4h:.6f}")

best_baseline_1h = min(naive_mae_1h, mean_mae_1h)
best_baseline_4h = min(naive_mae_4h, mean_mae_4h)

BASELINES = {
    '1h': {'naive_mae': naive_mae_1h, 'mean_mae': mean_mae_1h, 'best': best_baseline_1h},
    '4h': {'naive_mae': naive_mae_4h, 'mean_mae': mean_mae_4h, 'best': best_baseline_4h},
}

# Holdout baselines (CRITICAL for proper model selection)
if HAS_HOLDOUT:
    holdout_gas = df_holdout['gas']
    holdout_y_1h = df_holdout['target_1h']
    holdout_y_4h = df_holdout['target_4h']
    
    holdout_naive_1h = np.mean(np.abs(holdout_y_1h.values - holdout_gas.values))
    holdout_naive_4h = np.mean(np.abs(holdout_y_4h.values - holdout_gas.values))
    
    holdout_mean_pred = np.full_like(holdout_y_1h.values, y_1h.mean())  # Use training mean
    holdout_mean_1h = np.mean(np.abs(holdout_y_1h.values - holdout_mean_pred))
    holdout_mean_4h = np.mean(np.abs(holdout_y_4h.values - holdout_mean_pred))
    
    print(f"\nHOLDOUT SET Baseline MAEs:")
    print(f"  Naive (current price):     MAE_1h={holdout_naive_1h:.6f}, MAE_4h={holdout_naive_4h:.6f}")
    print(f"  Mean (historical average): MAE_1h={holdout_mean_1h:.6f}, MAE_4h={holdout_mean_4h:.6f}")
    
    BASELINES['1h']['holdout_naive_mae'] = holdout_naive_1h
    BASELINES['1h']['holdout_mean_mae'] = holdout_mean_1h
    BASELINES['1h']['holdout_best'] = min(holdout_naive_1h, holdout_mean_1h)
    
    BASELINES['4h']['holdout_naive_mae'] = holdout_naive_4h
    BASELINES['4h']['holdout_mean_mae'] = holdout_mean_4h
    BASELINES['4h']['holdout_best'] = min(holdout_naive_4h, holdout_mean_4h)

# === ADAPTIVE CONFIGURATION BASED ON VOLATILITY ===
# These will be used in Cell 7 for model training
ADAPTIVE_CONFIG = {
    'shift_detected': DISTRIBUTION_SHIFT_DETECTED,
    'shift_magnitude': SHIFT_MAGNITUDE,
    'volatility_level': volatility_level if DISTRIBUTION_SHIFT_DETECTED else 'NORMAL'
}

if DISTRIBUTION_SHIFT_DETECTED:
    print(f"\n{'='*60}")
    print("ADAPTIVE CONFIGURATION FOR VOLATILE PERIOD")
    print(f"{'='*60}")
    print(f"  Volatility level: {ADAPTIVE_CONFIG['volatility_level']}")
    print(f"  Shift magnitude: {SHIFT_MAGNITUDE:.2f}")
    print(f"  Rolling window: {ROLLING_WINDOW_DAYS} days")
    print("  → Model training will use relaxed requirements and increased regularization")

In [ ]:
# Model Training - WITH ADAPTIVE REGULARIZATION & RELAXED BASELINES FOR VOLATILE PERIODSfrom sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressorfrom sklearn.linear_model import Ridge, HuberRegressor, ElasticNetfrom sklearn.metrics import mean_absolute_error, mean_squared_error, r2_scorefrom sklearn.inspection import permutation_importanceimport joblibimport warningswarnings.filterwarnings('ignore')# === BASE CONFIGURATION ===TRAIN_REGIME_MODELS = TrueCOMPUTE_PERMUTATION_IMPORTANCE = TrueENABLE_FEATURE_PRUNING = TrueFEATURE_PRUNING_THRESHOLD = 0.005  # Remove features with < 0.5% importance (was 1%)MIN_FEATURES_TO_KEEP = 12  # NEW: Never prune below this many features for 1h# Ensemble settingsUSE_ENSEMBLE_BLENDING = TrueENSEMBLE_ASYM_WEIGHT = 0.6ASYMMETRIC_ALPHA = 0.6# === ADAPTIVE CONFIGURATION BASED ON VOLATILITY ===# Automatically adjust based on distribution shift detected in Cell 6if 'ADAPTIVE_CONFIG' in dir() and ADAPTIVE_CONFIG.get('shift_detected', False):    volatility = ADAPTIVE_CONFIG['volatility_level']    shift_mag = ADAPTIVE_CONFIG['shift_magnitude']        print(f"\n{'='*60}")    print(f"ADAPTIVE MODE: {volatility} VOLATILITY")    print(f"{'='*60}")        if volatility == 'SEVERE':        # SEVERE: Very relaxed requirements, maximum regularization        MINIMUM_IMPROVEMENT = -0.10        # Accept 10% WORSE than baseline        HOLDOUT_DEGRADATION_LIMIT = 0.80   # Allow 80% degradation        REGULARIZATION_STRENGTH = 0.5      # Strong regularization        MIN_SAMPLES_LEAF_MULT = 3.0        # Much larger leaves        print("  → Accepting models up to 10% worse than baseline")        print("  → Strong regularization (0.5), large leaf sizes (3x)")            elif volatility == 'HIGH':        # HIGH: Relaxed requirements, high regularization        MINIMUM_IMPROVEMENT = -0.05        # Accept 5% WORSE than baseline        HOLDOUT_DEGRADATION_LIMIT = 0.60   # Allow 60% degradation        REGULARIZATION_STRENGTH = 0.3      # Higher regularization        MIN_SAMPLES_LEAF_MULT = 2.5        # Larger leaves        print("  → Accepting models up to 5% worse than baseline")        print("  → High regularization (0.3), larger leaf sizes (2.5x)")            elif volatility == 'MODERATE':        # MODERATE: Slightly relaxed        MINIMUM_IMPROVEMENT = 0.0          # Just match baseline        HOLDOUT_DEGRADATION_LIMIT = 0.50   # Allow 50% degradation        REGULARIZATION_STRENGTH = 0.2      # Moderate regularization        MIN_SAMPLES_LEAF_MULT = 2.0        # Larger leaves        print("  → Accepting models that match baseline (0% improvement)")        print("  → Moderate regularization (0.2), leaf sizes (2x)")            else:  # MILD        MINIMUM_IMPROVEMENT = 0.02         # 2% improvement required        HOLDOUT_DEGRADATION_LIMIT = 0.40   # Allow 40% degradation        REGULARIZATION_STRENGTH = 0.15     # Light regularization        MIN_SAMPLES_LEAF_MULT = 1.75       # Slightly larger leaves        print("  → Requiring 2% improvement over baseline")        print("  → Light regularization (0.15), leaf sizes (1.75x)")else:    # NORMAL: Standard requirements    MINIMUM_IMPROVEMENT = 0.05    HOLDOUT_DEGRADATION_LIMIT = 0.30    REGULARIZATION_STRENGTH = 0.1    MIN_SAMPLES_LEAF_MULT = 1.5    print("\n[Standard mode - normal volatility requirements]")# === MODULE-LEVEL CLASS FOR PICKLING ===class EnsembleModel:    """Blended ensemble of symmetric and asymmetric models - defined at module level for pickling"""    def __init__(self, sym_model, asym_model, sym_scaler, asym_scaler, sym_weight, asym_weight):        self.sym_model = sym_model        self.asym_model = asym_model        self.sym_scaler = sym_scaler        self.asym_scaler = asym_scaler        self.sym_weight = sym_weight        self.asym_weight = asym_weight        def predict(self, X):        sym_pred = self.sym_model.predict(X)        asym_pred = self.asym_model.predict(X)        return self.sym_weight * sym_pred + self.asym_weight * asym_pred        def get_params(self, deep=True):        return {'sym_weight': self.sym_weight, 'asym_weight': self.asym_weight}def check_baseline_gate(model_mae, baseline_mae, model_name):    """Check if model beats baseline by minimum threshold (ADAPTIVE)"""    improvement = (baseline_mae - model_mae) / baseline_mae    passed = improvement >= MINIMUM_IMPROVEMENT        if passed:        if MINIMUM_IMPROVEMENT < 0:            print(f"  ✓ PASSED (relaxed): {improvement*100:.1f}% vs baseline (threshold: {MINIMUM_IMPROVEMENT*100:.0f}%)")        else:            print(f"  ✓ PASSED baseline gate: {improvement*100:.1f}% improvement")    else:        print(f"  ✗ FAILED baseline gate: {improvement*100:.1f}% (need {MINIMUM_IMPROVEMENT*100:.0f}%+)")    return passed, improvementdef check_holdout_gate(cv_mae, holdout_mae, model_name, holdout_baseline=None):    """Check if holdout performance is acceptable (ADAPTIVE)"""    if cv_mae <= 0:        return False, 0    degradation = (holdout_mae - cv_mae) / cv_mae        if holdout_baseline is not None:        holdout_improvement = (holdout_baseline - holdout_mae) / holdout_baseline        # During volatile periods, accept worse performance        threshold = MINIMUM_IMPROVEMENT        if holdout_improvement >= threshold:            print(f"  ✓ Beats holdout baseline by {holdout_improvement*100:.1f}% (threshold: {threshold*100:.0f}%)")            return True, degradation        passed = degradation < HOLDOUT_DEGRADATION_LIMIT    if passed:        print(f"  ✓ PASSED holdout gate: {degradation*100:+.1f}% degradation (limit: {HOLDOUT_DEGRADATION_LIMIT*100:.0f}%)")    else:        print(f"  ✗ FAILED holdout gate: {degradation*100:+.1f}% degradation (limit: {HOLDOUT_DEGRADATION_LIMIT*100:.0f}%)")    return passed, degradationdef asymmetric_loss(y_true, y_pred, alpha=0.6):    """Pinball loss - alpha > 0.5 penalizes under-prediction more"""    errors = y_true - y_pred    loss = np.where(errors >= 0, alpha * np.abs(errors), (1 - alpha) * np.abs(errors))    return np.mean(loss)def walk_forward_validate(model_class, model_params, X, y, baseline_mae, n_splits=5, purge_gap=120):    """Walk-forward validation with purge gap"""    n = len(X)    fold_size = n // (n_splits + 1)    fold_results = []        for fold in range(n_splits):        train_end = fold_size * (fold + 1)        test_start = train_end + purge_gap        test_end = test_start + fold_size                if test_end > n:            break                    X_train = X.iloc[:train_end]        X_test = X.iloc[test_start:test_end]        y_train = y.iloc[:train_end]        y_test = y.iloc[test_start:test_end]                scaler = RobustScaler()        X_train_scaled = scaler.fit_transform(X_train)        X_test_scaled = scaler.transform(X_test)                model = model_class(**model_params)        model.fit(X_train_scaled, y_train)                y_pred = model.predict(X_test_scaled)        mae = mean_absolute_error(y_test, y_pred)        fold_results.append(mae)        if not fold_results:        return None            return {        'avg_mae': np.mean(fold_results),        'std_mae': np.std(fold_results),        'improvement': (baseline_mae - np.mean(fold_results)) / baseline_mae    }def get_models_to_try():    """Get list of models with ADAPTIVE REGULARIZATION"""    # Scale regularization based on volatility    base_min_samples = int(30 * MIN_SAMPLES_LEAF_MULT)    ridge_alpha = 10.0 * (1 + REGULARIZATION_STRENGTH * 5)    huber_alpha = 0.5 * (1 + REGULARIZATION_STRENGTH * 3)    elastic_alpha = 0.1 * (1 + REGULARIZATION_STRENGTH * 3)    gbm_lr = max(0.03, 0.1 - REGULARIZATION_STRENGTH * 0.15)  # Slower learning with more regularization        models = [        ('Ridge', Ridge, {'alpha': ridge_alpha, 'random_state': 42}),        ('Huber', HuberRegressor, {'epsilon': 1.35, 'alpha': huber_alpha, 'max_iter': 1000}),        ('ElasticNet', ElasticNet, {'alpha': elastic_alpha, 'l1_ratio': 0.5, 'random_state': 42, 'max_iter': 2000}),        ('RF', RandomForestRegressor, {            'n_estimators': 100,  # More trees for stability            'max_depth': 4,       # Shallower trees            'min_samples_leaf': base_min_samples,            'max_features': 0.5,  # Use fewer features per tree for more robustness            'random_state': 42,             'n_jobs': -1        }),        ('GBM', GradientBoostingRegressor, {            'n_estimators': 80,             'max_depth': 3,      # Shallower            'learning_rate': gbm_lr,            'min_samples_leaf': base_min_samples,            'subsample': 0.7,    # More subsampling for regularization            'random_state': 42        }),        ('GBM-Asym', GradientBoostingRegressor, {            'loss': 'quantile',             'alpha': ASYMMETRIC_ALPHA,            'n_estimators': 80,             'max_depth': 3,            'learning_rate': gbm_lr,            'min_samples_leaf': base_min_samples,            'subsample': 0.7,            'random_state': 42        }),    ]        return modelsdef compute_permutation_importance(model, X, y, scaler, feature_names, n_repeats=5):    """Compute permutation importance for any model type"""    X_scaled = scaler.transform(X)        result = permutation_importance(        model, X_scaled, y,        n_repeats=n_repeats,        random_state=42,        scoring='neg_mean_absolute_error',        n_jobs=-1    )        importance_dict = {}    for i, feat in enumerate(feature_names):        importance_dict[feat] = -result.importances_mean[i]        total = sum(importance_dict.values())    if total > 0:        importance_dict = {k: v/total for k, v in importance_dict.items()}        return importance_dictdef prune_features_by_importance(X_train, X_holdout, feature_names, importance_dict, threshold=0.0):    """Remove features with importance below threshold"""    features_to_keep = [f for f in feature_names if importance_dict.get(f, 0) >= threshold]    features_to_remove = [f for f in feature_names if f not in features_to_keep]        if not features_to_remove:        return X_train, X_holdout, feature_names, []        print(f"  Feature pruning: removing {len(features_to_remove)} features with importance < {threshold}")    for f in features_to_remove[:5]:        print(f"    - {f}: {importance_dict.get(f, 0):.4f}")    if len(features_to_remove) > 5:        print(f"    ... and {len(features_to_remove) - 5} more")        X_train_pruned = X_train[features_to_keep]    X_holdout_pruned = X_holdout[features_to_keep]        return X_train_pruned, X_holdout_pruned, features_to_keep, features_to_removedef create_ensemble_model(sym_model, asym_model, sym_scaler, asym_scaler, sym_weight=0.4, asym_weight=0.6):    """Create blended ensemble of symmetric and asymmetric models"""    return EnsembleModel(sym_model, asym_model, sym_scaler, asym_scaler, sym_weight, asym_weight)def train_model_with_holdout(X_train, y_train, X_holdout, y_holdout, baseline_mae,                              horizon_name, feature_names, holdout_baseline=None):    """Train model with ADAPTIVE requirements"""    print(f"\n{'='*60}")    print(f"Training {horizon_name} model")    print(f"{'='*60}")    print(f"Train: {len(X_train):,}, Holdout: {len(X_holdout):,}, Features: {X_train.shape[1]}")    print(f"Train baseline: {baseline_mae:.6f}", end="")    if holdout_baseline:        print(f", Holdout baseline: {holdout_baseline:.6f}")    else:        print()        # Show adaptive settings    print(f"Adaptive settings: min_improvement={MINIMUM_IMPROVEMENT*100:.0f}%, "          f"reg_strength={REGULARIZATION_STRENGTH:.2f}, leaf_mult={MIN_SAMPLES_LEAF_MULT:.1f}x")        models_to_try = get_models_to_try()    results = []    sym_results = []    asym_results = []        for name, model_class, params in models_to_try:        print(f"\n[{name}]")        try:            wf_result = walk_forward_validate(model_class, params, X_train, y_train, baseline_mae, n_splits=4, purge_gap=120)            if not wf_result:                continue                            cv_mae = wf_result['avg_mae']            print(f"  CV MAE: {cv_mae:.6f} ± {wf_result['std_mae']:.6f}")                        scaler = RobustScaler()            X_train_scaled = scaler.fit_transform(X_train)            X_holdout_scaled = scaler.transform(X_holdout)                        model = model_class(**params)            model.fit(X_train_scaled, y_train)                        y_holdout_pred = model.predict(X_holdout_scaled)            holdout_mae = mean_absolute_error(y_holdout, y_holdout_pred)            holdout_improvement = (baseline_mae - holdout_mae) / baseline_mae                        asym_loss = asymmetric_loss(y_holdout.values, y_holdout_pred, ASYMMETRIC_ALPHA)                        if holdout_baseline:                vs_holdout = (holdout_baseline - holdout_mae) / holdout_baseline                print(f"  HOLDOUT MAE: {holdout_mae:.6f} ({vs_holdout*100:+.1f}% vs holdout baseline)")            else:                print(f"  HOLDOUT MAE: {holdout_mae:.6f} ({holdout_improvement*100:+.1f}% vs train baseline)")                        use_baseline = holdout_baseline if holdout_baseline else baseline_mae            passed_baseline, _ = check_baseline_gate(holdout_mae, use_baseline, name)            passed_holdout, degradation = check_holdout_gate(cv_mae, holdout_mae, name, holdout_baseline)                        result_entry = {                'name': name, 'model_class': model_class, 'params': params,                'cv_mae': cv_mae, 'holdout_mae': holdout_mae,                'asymmetric_loss': asym_loss,                'holdout_improvement': holdout_improvement,                'vs_holdout_baseline': (holdout_baseline - holdout_mae) / holdout_baseline if holdout_baseline else None,                'model': model, 'scaler': scaler            }                        # Relaxed acceptance during volatile periods            if passed_baseline or passed_holdout:                results.append(result_entry)                if 'Asym' in name:                    asym_results.append(result_entry)                else:                    sym_results.append(result_entry)                print(f"  → Accepted")            else:                print(f"  → Rejected")                        except Exception as e:            print(f"  Failed: {e}")        if not results:        print("\n⚠️ All models failed! Using Huber fallback...")        scaler = RobustScaler()        X_train_scaled = scaler.fit_transform(X_train)        model = HuberRegressor(epsilon=1.35, alpha=0.1 * (1 + REGULARIZATION_STRENGTH * 3), max_iter=1000)        model.fit(X_train_scaled, y_train)                y_holdout_pred = model.predict(scaler.transform(X_holdout))        holdout_mae = mean_absolute_error(y_holdout, y_holdout_pred)                importance = {}        if COMPUTE_PERMUTATION_IMPORTANCE:            importance = compute_permutation_importance(model, X_holdout, y_holdout, scaler, feature_names)                return model, scaler, {            'name': 'Huber (fallback)',            'mae': holdout_mae,            'improvement': (baseline_mae - holdout_mae) / baseline_mae,            'vs_holdout_baseline': (holdout_baseline - holdout_mae) / holdout_baseline if holdout_baseline else None,            'passed_baseline': False,            'is_fallback': True,            'is_ensemble': False        }, importance, list(feature_names)        # === ENSEMBLE BLENDING ===    best_single = min(results, key=lambda x: x['holdout_mae'])    best_model = best_single['model']    best_scaler = best_single['scaler']    best_metrics = {        'name': best_single['name'],        'mae': best_single['holdout_mae'],        'cv_mae': best_single['cv_mae'],        'asymmetric_loss': best_single.get('asymmetric_loss'),        'improvement': best_single['holdout_improvement'],        'vs_holdout_baseline': best_single['vs_holdout_baseline'],        'passed_baseline': True,        'is_fallback': False,        'is_ensemble': False    }        # Try ensemble if we have both symmetric and asymmetric models    if USE_ENSEMBLE_BLENDING and sym_results and asym_results:        print(f"\n>>> Trying ensemble blend...")        best_sym = min(sym_results, key=lambda x: x['holdout_mae'])        best_asym = min(asym_results, key=lambda x: x['holdout_mae'])                X_holdout_scaled = best_sym['scaler'].transform(X_holdout)        sym_pred = best_sym['model'].predict(X_holdout_scaled)        asym_pred = best_asym['model'].predict(X_holdout_scaled)                best_blend_mae = float('inf')        best_blend_weights = (0.4, 0.6)                for asym_w in [0.5, 0.55, 0.6, 0.65, 0.7]:            sym_w = 1 - asym_w            blend_pred = sym_w * sym_pred + asym_w * asym_pred            blend_mae = mean_absolute_error(y_holdout, blend_pred)            if blend_mae < best_blend_mae:                best_blend_mae = blend_mae                best_blend_weights = (sym_w, asym_w)                print(f"  Best single ({best_single['name']}): MAE={best_single['holdout_mae']:.6f}")        print(f"  Best ensemble ({best_sym['name']}+{best_asym['name']}): MAE={best_blend_mae:.6f}")        print(f"  Blend weights: {best_blend_weights[0]:.0%} sym + {best_blend_weights[1]:.0%} asym")                if best_blend_mae < best_single['holdout_mae']:            print(f"  ✓ Using ensemble (improves by {(best_single['holdout_mae'] - best_blend_mae) / best_single['holdout_mae'] * 100:.1f}%)")                        ensemble = create_ensemble_model(                best_sym['model'], best_asym['model'],                best_sym['scaler'], best_asym['scaler'],                best_blend_weights[0], best_blend_weights[1]            )                        best_model = ensemble            best_scaler = best_sym['scaler']            best_metrics = {                'name': f"Ensemble({best_sym['name']}+{best_asym['name']})",                'mae': best_blend_mae,                'cv_mae': (best_sym['cv_mae'] + best_asym['cv_mae']) / 2,                'improvement': (baseline_mae - best_blend_mae) / baseline_mae,                'vs_holdout_baseline': (holdout_baseline - best_blend_mae) / holdout_baseline if holdout_baseline else None,                'passed_baseline': True,                'is_fallback': False,                'is_ensemble': True,                'ensemble_weights': best_blend_weights,                'sym_model': best_sym['name'],                'asym_model': best_asym['name']            }        else:            print(f"  ✗ Single model is better, not using ensemble")        print(f"\n>>> Best: {best_metrics['name']} (Holdout MAE: {best_metrics['mae']:.6f})")        # Compute permutation importance    importance = {}    if COMPUTE_PERMUTATION_IMPORTANCE:        print("  Computing permutation importance...")        if best_metrics.get('is_ensemble'):            importance = compute_permutation_importance(                best_single['model'], X_holdout, y_holdout, best_scaler, list(feature_names)            )        else:            importance = compute_permutation_importance(                best_model, X_holdout, y_holdout, best_scaler, list(feature_names)            )        top_3 = sorted(importance.items(), key=lambda x: x[1], reverse=True)[:3]        print(f"  Top features: {', '.join([f'{f[0]}({f[1]:.2f})' for f in top_3])}")                # Feature pruning (only if not ensemble)        if ENABLE_FEATURE_PRUNING and not best_metrics.get('is_ensemble'):            negative_features = [f for f, v in importance.items() if v < FEATURE_PRUNING_THRESHOLD]            if negative_features and len(feature_names) - len(negative_features) >= 5:                X_train_pruned, X_holdout_pruned, pruned_features, removed = prune_features_by_importance(                    X_train, X_holdout, list(feature_names), importance, FEATURE_PRUNING_THRESHOLD                )                                if len(pruned_features) >= 5:                    print(f"  Re-training with {len(pruned_features)} features...")                                        scaler_pruned = RobustScaler()                    X_train_pruned_scaled = scaler_pruned.fit_transform(X_train_pruned)                    X_holdout_pruned_scaled = scaler_pruned.transform(X_holdout_pruned)                                        model_pruned = best_single['model_class'](**best_single['params'])                    model_pruned.fit(X_train_pruned_scaled, y_train)                                        y_holdout_pred_pruned = model_pruned.predict(X_holdout_pruned_scaled)                    holdout_mae_pruned = mean_absolute_error(y_holdout, y_holdout_pred_pruned)                                        print(f"  Pruned model MAE: {holdout_mae_pruned:.6f} (original: {best_metrics['mae']:.6f})")                                        if holdout_mae_pruned <= best_metrics['mae'] * 1.02:                        print(f"  ✓ Using pruned model ({len(pruned_features)} features)")                        best_model = model_pruned                        best_scaler = scaler_pruned                        best_metrics['mae'] = holdout_mae_pruned                        best_metrics['n_features_pruned'] = len(removed)                        feature_names = pruned_features                        importance = {f: importance[f] for f in pruned_features}        return best_model, best_scaler, best_metrics, importance, list(feature_names)def train_regime_models(X_train, y_train, X_holdout, y_holdout, regime_train, regime_holdout,                        baseline_mae, horizon_name, feature_names, holdout_baseline=None):    """Train separate models for each regime"""    print(f"\n{'='*60}")    print(f"Training REGIME-SPECIFIC {horizon_name} models")    print(f"{'='*60}")        regime_models = {}        for regime_val, regime_name in [(0, 'normal'), (1, 'elevated'), (2, 'spike')]:        train_mask = regime_train == regime_val        holdout_mask = regime_holdout == regime_val                n_train = train_mask.sum()        n_holdout = holdout_mask.sum()                print(f"\n[{regime_name.upper()}] Train: {n_train}, Holdout: {n_holdout}")                if n_train < 500 or n_holdout < 100:            print(f"  Insufficient data, skipping")            continue                X_r_train = X_train[train_mask]        y_r_train = y_train[train_mask]        X_r_holdout = X_holdout[holdout_mask]        y_r_holdout = y_holdout[holdout_mask]                model, scaler, metrics, importance, final_features = train_model_with_holdout(            X_r_train, y_r_train, X_r_holdout, y_r_holdout,            baseline_mae, f"{horizon_name}_{regime_name}", feature_names, holdout_baseline        )                if model:            regime_models[regime_val] = {                'model': model, 'scaler': scaler, 'metrics': metrics,                'regime_name': regime_name, 'n_samples': n_train,                'features': final_features            }        return regime_modelsdef print_distribution_diagnostics(y_train, y_holdout, name=""):    """Print diagnostics"""    print(f"\n[Distribution - {name}]")    print(f"  Train:   mean={y_train.mean():.4f}, std={y_train.std():.4f}")    print(f"  Holdout: mean={y_holdout.mean():.4f}, std={y_holdout.std():.4f}")

In [ ]:
# Train all models with ENSEMBLE REGIME SWITCHING
print("="*70)
print("TRAINING ALL PREDICTION MODELS")
print("="*70)

trained_models = {}
regime_specific_models = {}
all_feature_importance = {}
pruned_features_log = {}  # NEW: Track which features were pruned

if not HAS_HOLDOUT or df_holdout is None or len(df_holdout) < 1000:
    print("\n⚠️ WARNING: Limited holdout data")

print(f"\nTraining set: {len(df_train_val):,} samples")
print(f"Holdout set:  {len(df_holdout) if df_holdout is not None else 0:,} samples")
if DISTRIBUTION_SHIFT_DETECTED:
    print(f"⚠️ Distribution shift detected (magnitude: {SHIFT_MAGNITUDE:.2f})")
    if USE_ROLLING_WINDOW:
        print(f"   Auto-adapted to {ROLLING_WINDOW_DAYS}-day rolling window")

# === CREATE REGIME LABELS ===
print("\nCreating regime labels...")
regime_train = pd.Series(0, index=df_train_val.index)
if 'gas_zscore_1h' in df_train_val.columns:
    regime_train[df_train_val['gas_zscore_1h'] > 1] = 1
if 'is_spike' in df_train_val.columns:
    regime_train[df_train_val['is_spike'] == 1] = 2

regime_holdout = None
if HAS_HOLDOUT:
    regime_holdout = pd.Series(0, index=df_holdout.index)
    if 'gas_zscore_1h' in df_holdout.columns:
        regime_holdout[df_holdout['gas_zscore_1h'] > 1] = 1
    if 'is_spike' in df_holdout.columns:
        regime_holdout[df_holdout['is_spike'] == 1] = 2

print(f"Regime distribution (train): {dict(regime_train.value_counts().sort_index())}")
if regime_holdout is not None:
    print(f"Regime distribution (holdout): {dict(regime_holdout.value_counts().sort_index())}")

# === ENSEMBLE PREDICTION FUNCTION ===
def create_ensemble_predictor(global_model, global_scaler, regime_models, features):
    """Create a predictor that uses regime-specific models when available.
    Handles feature mismatches when regime models use different (pruned) features.
    """
    def predict(X, current_regime=None):
        # Handle both DataFrame and array inputs
        if hasattr(X, 'columns'):
            # DataFrame input - select features by name
            X_global = X[features] if all(f in X.columns for f in features) else X
            X_scaled = global_scaler.transform(X_global)
        else:
            # Array input - assume correct feature order
            X_scaled = global_scaler.transform(X)
        
        global_pred = global_model.predict(X_scaled)
        
        if current_regime is not None and regime_models and current_regime in regime_models:
            regime_data = regime_models[current_regime]
            regime_features = regime_data.get('features', features)
            
            if hasattr(X, 'columns'):
                # DataFrame - select regime-specific features
                available_features = [f for f in regime_features if f in X.columns]
                if len(available_features) == len(regime_features):
                    X_regime = X[regime_features]
                    X_regime_scaled = regime_data['scaler'].transform(X_regime)
                    regime_pred = regime_data['model'].predict(X_regime_scaled)
                    # Weighted average: 70% regime, 30% global
                    return 0.7 * regime_pred + 0.3 * global_pred
            else:
                # Array input - only use if feature counts match
                if X.shape[1] == len(regime_features):
                    X_regime_scaled = regime_data['scaler'].transform(X)
                    regime_pred = regime_data['model'].predict(X_regime_scaled)
                    return 0.7 * regime_pred + 0.3 * global_pred
        
        return global_pred
    
    return predict

# === 1H MODEL ===
print("\n" + "="*70)
print("1-HOUR MODEL")
print("="*70)

X_holdout_1h = df_holdout[numeric_features_1h] if HAS_HOLDOUT else X_1h.iloc[-1000:]
y_holdout_1h = df_holdout['target_1h'] if HAS_HOLDOUT else y_1h.iloc[-1000:]
mask_1h = y_holdout_1h.notna()
X_holdout_1h = X_holdout_1h[mask_1h]
y_holdout_1h = y_holdout_1h[mask_1h]

print_distribution_diagnostics(y_1h, y_holdout_1h, "1h targets")

holdout_baseline_1h = BASELINES['1h'].get('holdout_best', None)

# Updated: now returns 5 values (model, scaler, metrics, importance, final_features)
model_1h, scaler_1h, metrics_1h, importance_1h, features_1h = train_model_with_holdout(
    X_1h, y_1h, X_holdout_1h, y_holdout_1h,
    BASELINES['1h']['best'], '1h', list(numeric_features_1h), holdout_baseline_1h
)
if model_1h:
    trained_models['1h'] = {
        'model': model_1h, 'scaler': scaler_1h, 
        'metrics': metrics_1h, 'features': features_1h  # Use possibly-pruned features
    }
    if importance_1h:
        all_feature_importance['1h'] = importance_1h
    
    # Log if features were pruned
    if len(features_1h) < len(numeric_features_1h):
        pruned_features_log['1h'] = {
            'original_count': len(numeric_features_1h),
            'pruned_count': len(features_1h),
            'removed': list(set(numeric_features_1h) - set(features_1h))
        }
        print(f"  ✓ Features pruned: {len(numeric_features_1h)} → {len(features_1h)}")

# Train regime-specific models
if TRAIN_REGIME_MODELS and regime_holdout is not None:
    regime_holdout_1h = regime_holdout[mask_1h]
    regime_models_1h = train_regime_models(
        X_1h, y_1h, X_holdout_1h, y_holdout_1h,
        regime_train, regime_holdout_1h,
        BASELINES['1h']['best'], '1h', list(numeric_features_1h), holdout_baseline_1h
    )
    if regime_models_1h:
        regime_specific_models['1h'] = regime_models_1h
        # Create ensemble predictor with the actual features used
        trained_models['1h']['ensemble_predict'] = create_ensemble_predictor(
            model_1h, scaler_1h, regime_models_1h, features_1h
        )

# === 4H MODEL ===
print("\n" + "="*70)
print("4-HOUR MODEL")
print("="*70)

X_holdout_4h = df_holdout[numeric_features_4h] if HAS_HOLDOUT else X_4h.iloc[-1000:]
y_holdout_4h = df_holdout['target_4h'] if HAS_HOLDOUT else y_4h.iloc[-1000:]
mask_4h = y_holdout_4h.notna()
X_holdout_4h = X_holdout_4h[mask_4h]
y_holdout_4h = y_holdout_4h[mask_4h]

print_distribution_diagnostics(y_4h, y_holdout_4h, "4h targets")

holdout_baseline_4h = BASELINES['4h'].get('holdout_best', None)

model_4h, scaler_4h, metrics_4h, importance_4h, features_4h = train_model_with_holdout(
    X_4h, y_4h, X_holdout_4h, y_holdout_4h,
    BASELINES['4h']['best'], '4h', list(numeric_features_4h), holdout_baseline_4h
)
if model_4h:
    trained_models['4h'] = {
        'model': model_4h, 'scaler': scaler_4h,
        'metrics': metrics_4h, 'features': features_4h
    }
    if importance_4h:
        all_feature_importance['4h'] = importance_4h
    
    if len(features_4h) < len(numeric_features_4h):
        pruned_features_log['4h'] = {
            'original_count': len(numeric_features_4h),
            'pruned_count': len(features_4h),
            'removed': list(set(numeric_features_4h) - set(features_4h))
        }
        print(f"  ✓ Features pruned: {len(numeric_features_4h)} → {len(features_4h)}")

if TRAIN_REGIME_MODELS and regime_holdout is not None:
    regime_holdout_4h = regime_holdout[mask_4h]
    regime_models_4h = train_regime_models(
        X_4h, y_4h, X_holdout_4h, y_holdout_4h,
        regime_train, regime_holdout_4h,
        BASELINES['4h']['best'], '4h', list(numeric_features_4h), holdout_baseline_4h
    )
    if regime_models_4h:
        regime_specific_models['4h'] = regime_models_4h
        trained_models['4h']['ensemble_predict'] = create_ensemble_predictor(
            model_4h, scaler_4h, regime_models_4h, features_4h
        )

# === 24H MODEL ===
print("\n" + "="*70)
print("24-HOUR MODEL")
print("="*70)

rph = 120
total_hours = len(df_clean) / rph
total_days = total_hours / 24
print(f"Total data: {total_days:.1f} days")

if total_days >= 30:
    mask_24h_train = y_24h.notna()
    X_24h_valid = X_24h[mask_24h_train]
    y_24h_valid = y_24h[mask_24h_train]
    
    if HAS_HOLDOUT:
        y_holdout_24h = df_holdout['target_24h']
        mask_24h_holdout = y_holdout_24h.notna()
        X_holdout_24h = df_holdout[numeric_features_24h][mask_24h_holdout]
        y_holdout_24h = y_holdout_24h[mask_24h_holdout]
    else:
        X_holdout_24h = X_24h_valid.iloc[-500:]
        y_holdout_24h = y_24h_valid.iloc[-500:]
    
    if len(y_holdout_24h) > 100:
        model_24h, scaler_24h, metrics_24h, _, features_24h = train_model_with_holdout(
            X_24h_valid, y_24h_valid, X_holdout_24h, y_holdout_24h,
            BASELINES['4h']['best'], '24h', list(numeric_features_24h)
        )
        if model_24h:
            trained_models['24h'] = {
                'model': model_24h, 'scaler': scaler_24h,
                'metrics': metrics_24h, 'features': features_24h,
                'is_fallback': False
            }
    else:
        print(f"⚠️ Using 4h model as 24h fallback")
        if model_4h:
            trained_models['24h'] = {
                'model': model_4h, 'scaler': scaler_4h,
                'metrics': {'name': metrics_4h['name'] + ' (4h fallback)', 'mae': metrics_4h['mae'],
                           'improvement': metrics_4h['improvement'], 
                           'vs_holdout_baseline': metrics_4h.get('vs_holdout_baseline'),
                           'passed_baseline': metrics_4h.get('passed_baseline', False)},
                'features': features_4h,
                'is_fallback': True
            }
else:
    print(f"⚠️ Using 4h model as 24h fallback ({total_days:.1f} days < 30)")
    if model_4h:
        trained_models['24h'] = {
            'model': model_4h, 'scaler': scaler_4h,
            'metrics': {'name': metrics_4h['name'] + ' (4h fallback)', 'mae': metrics_4h['mae'],
                       'improvement': metrics_4h['improvement'],
                       'vs_holdout_baseline': metrics_4h.get('vs_holdout_baseline'),
                       'passed_baseline': metrics_4h.get('passed_baseline', False)},
            'features': features_4h,
            'is_fallback': True
        }

# === SUMMARY ===
print(f"\n{'='*70}")
print("TRAINING SUMMARY")
print(f"{'='*70}")

for horizon, data in trained_models.items():
    m = data['metrics']
    status = "✓" if m.get('passed_baseline', False) else "⚠"
    fallback = " (fallback)" if data.get('is_fallback') else ""
    
    # Show vs holdout baseline if available
    if m.get('vs_holdout_baseline') is not None:
        vs_baseline = f"{m['vs_holdout_baseline']*100:+.1f}% vs holdout baseline"
    else:
        vs_baseline = f"{m['improvement']*100:+.1f}% vs train baseline"
    
    has_ensemble = " [+ensemble]" if 'ensemble_predict' in data else ""
    n_features = len(data.get('features', []))
    print(f"{status} {horizon}: {m['name']}{fallback} | MAE: {m['mae']:.4f} | {vs_baseline} | {n_features} features{has_ensemble}")

if regime_specific_models:
    print(f"\nRegime-specific models:")
    for horizon, regime_dict in regime_specific_models.items():
        for regime_val, regime_data in regime_dict.items():
            print(f"  {horizon}_{regime_data['regime_name']}: MAE={regime_data['metrics']['mae']:.4f}")

if pruned_features_log:
    print(f"\nFeature pruning summary:")
    for horizon, log in pruned_features_log.items():
        print(f"  {horizon}: {log['original_count']} → {log['pruned_count']} features")

FEATURE_IMPORTANCE = all_feature_importance.get('4h', all_feature_importance.get('1h', {}))

In [ ]:
# TIME-OF-DAY SPECIFIC MODELS
# Train lightweight models for each time period (afternoon has 2x higher errors)

print("\n" + "="*60)
print("TIME-OF-DAY SPECIFIC MODELS")
print("="*60)

TRAIN_TIME_SPECIFIC_MODELS = True
MIN_SAMPLES_PER_PERIOD = 500

time_specific_models = {}

if TRAIN_TIME_SPECIFIC_MODELS:
    time_periods = {
        'night': (0, 6),
        'morning': (6, 12),
        'afternoon': (12, 18),  # Highest errors
        'evening': (18, 24)
    }
    
    for horizon in ['1h', '4h']:
        if horizon not in trained_models:
            continue
            
        print(f"\n{horizon} Time-Specific Models:")
        
        features = trained_models[horizon]['features']
        global_model = trained_models[horizon]['model']
        global_scaler = trained_models[horizon]['scaler']
        global_mae = trained_models[horizon]['metrics']['mae']
        
        time_specific_models[horizon] = {}
        
        for period_name, (start_hour, end_hour) in time_periods.items():
            # Filter data for this time period
            train_hours = df_train_val.index.hour
            train_mask = (train_hours >= start_hour) & (train_hours < end_hour)
            
            X_train_period = df_train_val.loc[train_mask, features]
            y_train_period = df_train_val.loc[train_mask, f'target_{horizon}']
            
            # Remove NaN
            valid_mask = y_train_period.notna()
            X_train_period = X_train_period[valid_mask]
            y_train_period = y_train_period[valid_mask]
            
            if len(X_train_period) < MIN_SAMPLES_PER_PERIOD:
                print(f"  {period_name}: Insufficient data ({len(X_train_period)} samples)")
                continue
            
            # Holdout data for this period
            if HAS_HOLDOUT:
                holdout_hours = df_holdout.index.hour
                holdout_mask = (holdout_hours >= start_hour) & (holdout_hours < end_hour)
                
                X_holdout_period = df_holdout.loc[holdout_mask, features]
                y_holdout_period = df_holdout.loc[holdout_mask, f'target_{horizon}']
                
                valid_mask_h = y_holdout_period.notna()
                X_holdout_period = X_holdout_period[valid_mask_h]
                y_holdout_period = y_holdout_period[valid_mask_h]
            else:
                # Split training data
                split_idx = int(len(X_train_period) * 0.8)
                X_holdout_period = X_train_period.iloc[split_idx:]
                y_holdout_period = y_train_period.iloc[split_idx:]
                X_train_period = X_train_period.iloc[:split_idx]
                y_train_period = y_train_period.iloc[:split_idx]
            
            if len(X_holdout_period) < 50:
                print(f"  {period_name}: Insufficient holdout data")
                continue
            
            # Train a simple model for this period
            scaler = RobustScaler()
            X_train_scaled = scaler.fit_transform(X_train_period)
            X_holdout_scaled = scaler.transform(X_holdout_period)
            
            # Use GBM with moderate complexity
            model = GradientBoostingRegressor(
                n_estimators=30, max_depth=4, learning_rate=0.1,
                min_samples_leaf=30, subsample=0.8, random_state=42
            )
            model.fit(X_train_scaled, y_train_period)
            
            # Evaluate
            y_pred = model.predict(X_holdout_scaled)
            period_mae = mean_absolute_error(y_holdout_period, y_pred)
            
            # Compare to global model
            global_pred = global_model.predict(global_scaler.transform(X_holdout_period))
            global_period_mae = mean_absolute_error(y_holdout_period, global_pred)
            
            improvement = (global_period_mae - period_mae) / global_period_mae * 100
            
            print(f"  {period_name}: Global MAE={global_period_mae:.4f}, Period MAE={period_mae:.4f} ({improvement:+.1f}%)")
            
            # Only use period-specific model if it's better
            if period_mae < global_period_mae:
                time_specific_models[horizon][period_name] = {
                    'model': model,
                    'scaler': scaler,
                    'mae': float(period_mae),
                    'global_mae': float(global_period_mae),
                    'improvement': float(improvement),
                    'n_samples': len(X_train_period),
                    'features': features
                }
                print(f"    ✓ Using period-specific model")
            else:
                print(f"    ✗ Global model is better, skipping")
        
        # Create time-adaptive predictor
        if time_specific_models.get(horizon):
            def create_time_adaptive_predictor(global_model, global_scaler, time_models, features):
                def predict(X, hour=None):
                    if hour is None:
                        # Use global
                        X_scaled = global_scaler.transform(X[features] if hasattr(X, 'columns') else X)
                        return global_model.predict(X_scaled)
                    
                    # Determine period
                    if 0 <= hour < 6:
                        period = 'night'
                    elif 6 <= hour < 12:
                        period = 'morning'
                    elif 12 <= hour < 18:
                        period = 'afternoon'
                    else:
                        period = 'evening'
                    
                    # Use period-specific model if available
                    if period in time_models:
                        tm = time_models[period]
                        X_scaled = tm['scaler'].transform(X[features] if hasattr(X, 'columns') else X)
                        return tm['model'].predict(X_scaled)
                    else:
                        # Fall back to global
                        X_scaled = global_scaler.transform(X[features] if hasattr(X, 'columns') else X)
                        return global_model.predict(X_scaled)
                
                return predict
            
            trained_models[horizon]['time_adaptive_predict'] = create_time_adaptive_predictor(
                global_model, global_scaler, time_specific_models[horizon], features
            )
            
            print(f"\n  ✓ Time-adaptive predictor created with {len(time_specific_models[horizon])} period models")

# Summary
print(f"\n{'='*60}")
print("TIME-SPECIFIC MODELS SUMMARY")
print("="*60)

for horizon, periods in time_specific_models.items():
    if periods:
        print(f"\n{horizon}:")
        for period, data in periods.items():
            print(f"  {period}: MAE={data['mae']:.4f} ({data['improvement']:+.1f}% vs global)")
    else:
        print(f"\n{horizon}: No period-specific models (global is best)")

print(f"\n✓ Time-of-day model training complete")

In [ ]:
# PREDICTION INTERVALS - FIXED CALIBRATION on HOLDOUT DATA# IMPROVED: Night-specific asymmetric intervals + Tail risk cappingfrom sklearn.ensemble import GradientBoostingRegressor, IsolationForestprint("\n" + "="*60)print("TRAINING PREDICTION INTERVALS (HOLDOUT-CALIBRATED)")print("="*60)quantile_models = {}conformal_residuals = {}uncertainty_scalers = {}time_period_calibration = {}tail_risk_config = {}  # NEW: Store tail risk capping configdef compute_holdout_conformal_interval(X_holdout, y_holdout, model, scaler, alpha=0.2):    """    Compute conformal interval on HOLDOUT data for proper calibration.    alpha=0.2 means 80% interval.    """    X_scaled = scaler.transform(X_holdout)    y_pred = model.predict(X_scaled)    residuals = np.abs(y_holdout.values - y_pred)    # Use holdout residuals directly for calibration    q = np.quantile(residuals, 1 - alpha)    # Verify coverage on holdout    coverage = np.mean(residuals <= q)    return {        'quantile': q,        'residuals': residuals,        'coverage_target': 1 - alpha,        'actual_coverage': coverage,        'calibration_source': 'holdout'    }def compute_adaptive_conformal_intervals_v2(X_holdout, y_holdout, model, scaler, hours, regimes=None):    """    IMPROVED: Compute separate conformal intervals with asymmetric handling for night.    Ensures 80% coverage across all conditions including problematic night period.    """    X_scaled = scaler.transform(X_holdout)    y_pred = model.predict(X_scaled)    residuals = np.abs(y_holdout.values - y_pred)    signed_residuals = y_holdout.values - y_pred  # For asymmetric analysis    # Time-period specific intervals    time_intervals = {}    time_periods = {        'night': (0, 6),        'morning': (6, 12),        'afternoon': (12, 18),        'evening': (18, 24)    }    overall_q80 = np.quantile(residuals, 0.8)    for period_name, (start, end) in time_periods.items():        mask = (hours >= start) & (hours < end)        if mask.sum() >= 50:            period_residuals = residuals[mask]            period_signed = signed_residuals[mask]            q80 = np.quantile(period_residuals, 0.8)            q90 = np.quantile(period_residuals, 0.9)            # NEW: Asymmetric intervals for night (spikes tend to be upward)            # Calculate separate up/down intervals            up_residuals = period_signed[period_signed > 0]  # Under-predictions            down_residuals = np.abs(period_signed[period_signed < 0])  # Over-predictions            # For night: use wider upside interval (gas can spike up)            if period_name == 'night' and len(up_residuals) > 10:                up_q80 = np.quantile(up_residuals, 0.85) if len(up_residuals) > 10 else q80                down_q80 = np.quantile(down_residuals, 0.75) if len(down_residuals) > 10 else q80 * 0.7                # If night coverage is poor, widen further                actual_coverage = np.mean(period_residuals <= q80)                if actual_coverage < 0.75:                    coverage_shortfall = (0.80 - actual_coverage) / 0.80                    q80 = q80 * (1 + coverage_shortfall * 0.5)  # Widen by up to 50%                    q90 = q90 * (1 + coverage_shortfall * 0.5)                    up_q80 = up_q80 * (1 + coverage_shortfall * 0.3)                    print(f"  Night coverage fix: widening intervals by {coverage_shortfall*50:.0f}%")            else:                up_q80 = q80                down_q80 = q80            multiplier = q80 / overall_q80 if overall_q80 > 0 else 1.0            time_intervals[period_name] = {                'interval_80': float(q80),                'interval_90': float(q90),                'interval_up_80': float(up_q80),    # NEW: Asymmetric up                'interval_down_80': float(down_q80),  # NEW: Asymmetric down                'multiplier': float(max(multiplier, 1.0)),                'mae': float(np.mean(period_residuals)),                'n_samples': int(mask.sum()),                'actual_coverage_80': float(np.mean(period_residuals <= q80)),                'is_asymmetric': period_name == 'night'  # Flag for asymmetric intervals            }    # Regime-specific intervals    regime_intervals = {}    if regimes is not None:        for regime_val, regime_name in [(0, 'normal'), (1, 'elevated'), (2, 'spike')]:            mask = regimes == regime_val            if mask.sum() >= 50:                regime_residuals = residuals[mask]                q80 = np.quantile(regime_residuals, 0.8)                q90 = np.quantile(regime_residuals, 0.9)                multiplier = q80 / overall_q80 if overall_q80 > 0 else 1.0                regime_intervals[regime_name] = {                    'interval_80': float(q80),                    'interval_90': float(q90),                    'multiplier': float(max(multiplier, 1.0)),                    'mae': float(np.mean(regime_residuals)),                    'n_samples': int(mask.sum()),                    'actual_coverage_80': float(np.mean(regime_residuals <= q80))                }    return time_intervals, regime_intervalsdef compute_tail_risk_caps(y_true, y_pred, percentile=0.95):    """    NEW: Compute prediction capping values to handle tail risk.    Returns caps that limit extreme predictions.    """    errors = y_true - y_pred    # Cap extreme predictions based on historical distribution    pred_min = np.percentile(y_true, 1)    pred_max = np.percentile(y_true, 99)    # Error-based caps: limit to P95 error magnitude    error_p95 = np.percentile(np.abs(errors), percentile)    # Range-based cap: limit change from recent mean    recent_mean = np.mean(y_true[-min(500, len(y_true)):])    recent_std = np.std(y_true[-min(500, len(y_true)):])    return {        'pred_min': float(pred_min),        'pred_max': float(pred_max),        'error_p95': float(error_p95),        'recent_mean': float(recent_mean),        'recent_std': float(recent_std),        'max_deviation_from_mean': float(3.0 * recent_std),  # 3 sigma limit        'winsorize_percentile': percentile    }def apply_tail_risk_capping(predictions, caps, current_gas=None):    """    Apply tail risk capping to predictions.    Prevents extreme predictions that are unlikely to be correct.    """    capped = np.array(predictions).copy()    # Method 1: Hard min/max from historical    capped = np.clip(capped, caps['pred_min'], caps['pred_max'])    # Method 2: Limit deviation from recent mean    if current_gas is not None:        max_up = current_gas + caps['max_deviation_from_mean']        max_down = current_gas - caps['max_deviation_from_mean']        max_down = max(max_down, caps['pred_min'])  # Don't go below historical min        capped = np.clip(capped, max_down, max_up)    return cappeddef train_uncertainty_scaler(X_train, scaler, residuals, features):    """Train OOD detector for uncertainty scaling"""    X_scaled = scaler.transform(X_train)    iso_forest = IsolationForest(        n_estimators=50, contamination=0.1,        random_state=42, n_jobs=-1    )    iso_forest.fit(X_scaled)    feature_means = X_scaled.mean(axis=0)    feature_stds = X_scaled.std(axis=0) + 1e-8    base_interval = np.quantile(residuals, 0.8)    return {        'iso_forest': iso_forest,        'feature_means': feature_means,        'feature_stds': feature_stds,        'base_interval': base_interval,        'features': features    }def get_adaptive_interval_v2(base_interval, hour=None, regime=None,                             time_intervals=None, regime_intervals=None,                             direction='symmetric'):    """    IMPROVED: Get the appropriate interval width with asymmetric support.    direction: 'up', 'down', or 'symmetric'    """    multiplier = 1.0    # Time-based adjustment    if hour is not None and time_intervals:        period = None        if 0 <= hour < 6:            period = 'night'        elif 6 <= hour < 12:            period = 'morning'        elif 12 <= hour < 18:            period = 'afternoon'        else:            period = 'evening'        if period in time_intervals:            ti = time_intervals[period]            if ti.get('is_asymmetric', False) and direction != 'symmetric':                # Use asymmetric intervals for night                if direction == 'up':                    return ti.get('interval_up_80', ti['interval_80'])                else:                    return ti.get('interval_down_80', ti['interval_80'])            multiplier = max(multiplier, ti['multiplier'])    # Regime-based adjustment    if regime is not None and regime_intervals:        regime_name = {0: 'normal', 1: 'elevated', 2: 'spike'}.get(regime)        if regime_name in regime_intervals:            multiplier = max(multiplier, regime_intervals[regime_name]['multiplier'])    return base_interval * multiplierfor horizon in ['1h', '4h']:    if horizon not in trained_models:        continue    print(f"\n{horizon} prediction intervals...")    data = trained_models[horizon]    model = data['model']    scaler = data['scaler']    features = data['features']    # === Use HOLDOUT data for calibration ===    if not HAS_HOLDOUT:        print(f"  ⚠️ No holdout data, using training data (may be miscalibrated)")        X_cal = df_train_val[features]        y_cal = df_train_val[f'target_{horizon}']    else:        X_cal = df_holdout[features]        y_cal = df_holdout[f'target_{horizon}']    mask = y_cal.notna()    X_cal = X_cal[mask]    y_cal = y_cal[mask]    if len(X_cal) < 500:        print(f"  ⚠️ Insufficient data for {horizon} intervals, skipping")        continue    # Get hours and regimes for adaptive calibration    hours = X_cal.index.hour if hasattr(X_cal.index, 'hour') else pd.Series(12, index=X_cal.index)    regimes = pd.Series(0, index=X_cal.index)    if 'gas_zscore_1h' in df_holdout.columns if HAS_HOLDOUT else df_train_val.columns:        source_df = df_holdout if HAS_HOLDOUT else df_train_val        regimes[source_df.loc[X_cal.index, 'gas_zscore_1h'] > 1] = 1    if 'is_spike' in df_holdout.columns if HAS_HOLDOUT else df_train_val.columns:        source_df = df_holdout if HAS_HOLDOUT else df_train_val        regimes[source_df.loc[X_cal.index, 'is_spike'] == 1] = 2    # === Compute HOLDOUT-CALIBRATED conformal intervals ===    conformal = compute_holdout_conformal_interval(X_cal, y_cal, model, scaler, alpha=0.2)    conformal_residuals[horizon] = conformal    print(f"  ✓ Conformal interval (holdout-calibrated): ±{conformal['quantile']:.4f}")    print(f"    Actual coverage on holdout: {conformal['actual_coverage']:.1%}")    # === Compute ADAPTIVE intervals with ASYMMETRIC night support ===    time_intervals, regime_intervals = compute_adaptive_conformal_intervals_v2(        X_cal, y_cal, model, scaler, hours.values, regimes.values    )    time_period_calibration[horizon] = {        'time': time_intervals,        'regime': regime_intervals    }    print(f"  ✓ Time-adaptive intervals computed:")    for period, info in time_intervals.items():        asym_flag = " [ASYMMETRIC]" if info.get('is_asymmetric', False) else ""        print(f"    {period}: ±{info['interval_80']:.4f} (coverage: {info['actual_coverage_80']:.1%}){asym_flag}")        if info.get('is_asymmetric', False):            print(f"      Up: {info.get('interval_up_80', 0):.4f}, Down: {info.get('interval_down_80', 0):.4f}")    # === NEW: Compute tail risk caps ===    X_scaled = scaler.transform(X_cal)    y_pred = model.predict(X_scaled)    tail_caps = compute_tail_risk_caps(y_cal.values, y_pred, percentile=0.95)    tail_risk_config[horizon] = tail_caps    print(f"  ✓ Tail risk caps: pred range [{tail_caps['pred_min']:.2f}, {tail_caps['pred_max']:.2f}]")    print(f"    Max deviation from mean: ±{tail_caps['max_deviation_from_mean']:.2f}")    # === Train quantile models on training data ===    X_train_q = df_train_val[features]    y_train_q = df_train_val[f'target_{horizon}']    mask_q = y_train_q.notna()    X_train_q = X_train_q[mask_q]    y_train_q = y_train_q[mask_q]    split_idx = int(len(X_train_q) * 0.8)    X_train, X_test = X_train_q.iloc[:split_idx], X_train_q.iloc[split_idx:]    y_train, y_test = y_train_q.iloc[:split_idx], y_train_q.iloc[split_idx:]    q_scaler = RobustScaler()    X_train_scaled = q_scaler.fit_transform(X_train)    q_models = {}    for q in [0.1, 0.5, 0.9]:        qmodel = GradientBoostingRegressor(            loss='quantile', alpha=q,            n_estimators=50, max_depth=4,            learning_rate=0.1, random_state=42        )        qmodel.fit(X_train_scaled, y_train)        q_models[q] = qmodel    quantile_models[horizon] = (q_models, q_scaler)    print(f"  ✓ Quantile models trained")    # === Train uncertainty scaler ===    unc_scaler = train_uncertainty_scaler(X_train, q_scaler, conformal['residuals'], features)    uncertainty_scalers[horizon] = unc_scaler    print(f"  ✓ Uncertainty scaler trained")    # === Verify calibration on holdout ===    abs_errors = np.abs(y_cal.values - y_pred)    conf_coverage = np.mean(abs_errors <= conformal['quantile'])    # Adaptive coverage (using time/regime specific intervals)    adaptive_coverages = []    for i, (idx, row) in enumerate(X_cal.iterrows()):        h = hours.iloc[i] if hasattr(hours, 'iloc') else hours[i]        r = regimes.iloc[i] if hasattr(regimes, 'iloc') else regimes[i]        adaptive_interval = get_adaptive_interval_v2(            conformal['quantile'], hour=h, regime=r,            time_intervals=time_intervals, regime_intervals=regime_intervals        )        adaptive_coverages.append(abs_errors[i] <= adaptive_interval)    adaptive_coverage = np.mean(adaptive_coverages)    print(f"\n  Calibration Verification (on holdout):")    print(f"    Conformal 80% interval: actual = {conf_coverage:.1%}")    print(f"    Adaptive 80% interval: actual = {adaptive_coverage:.1%}")    # Check night specifically    night_mask = (hours.values >= 0) & (hours.values < 6)    if night_mask.sum() > 50:        night_coverage = np.mean(np.array(adaptive_coverages)[night_mask])        print(f"    Night period coverage: {night_coverage:.1%}")        if night_coverage < 0.75:            print(f"    ⚠️ Night coverage still low - may need wider intervals")    # Store calibration in model data    trained_models[horizon]['calibration'] = {        'conformal_coverage': float(conf_coverage),        'adaptive_coverage': float(adaptive_coverage),        'conformal_width': float(conformal['quantile']),        'time_intervals': time_intervals,        'regime_intervals': regime_intervals,        'calibration_source': 'holdout',        'tail_risk_caps': tail_caps  # NEW: Include tail risk config    }    # Store warnings    warnings_list = []    for period, info in time_intervals.items():        if info['actual_coverage_80'] < 0.75:            warnings_list.append(f"{period} period under-covered: {info['actual_coverage_80']:.1%} (target 80%)")    trained_models[horizon]['calibration']['warnings'] = warnings_list# Copy 4h to 24hif '4h' in quantile_models:    quantile_models['24h'] = quantile_models['4h']    print("\n24h: Using 4h quantile models")if '4h' in conformal_residuals:    conformal_residuals['24h'] = conformal_residuals['4h']if '4h' in uncertainty_scalers:    uncertainty_scalers['24h'] = uncertainty_scalers['4h']if '4h' in time_period_calibration:    time_period_calibration['24h'] = time_period_calibration['4h']if '4h' in tail_risk_config:    tail_risk_config['24h'] = tail_risk_config['4h']print(f"\n✓ Prediction intervals (holdout-calibrated) ready for: {list(quantile_models.keys())}")print(f"✓ Tail risk capping configured for: {list(tail_risk_config.keys())}")

In [ ]:
# MULTI-REGIME CALIBRATION VERIFICATION
# Verify that prediction intervals are well-calibrated across different conditions

print("\n" + "="*60)
print("MULTI-REGIME CALIBRATION VERIFICATION")
print("="*60)

regime_calibration = {}

def verify_calibration_by_regime(X, y, model, scaler, conformal_quantile, 
                                  regime_labels, time_calibration=None, 
                                  uncertainty_scaler=None):
    """
    Verify prediction interval coverage for each regime and time period.
    Returns detailed calibration metrics.
    """
    X_scaled = scaler.transform(X)
    y_pred = model.predict(X_scaled)
    errors = np.abs(y.values - y_pred)
    
    results = {
        'overall': {},
        'by_regime': {},
        'by_time': {},
        'warnings': []
    }
    
    # Overall coverage at different interval widths
    for coverage_target in [0.8, 0.9, 0.95]:
        interval_width = np.quantile(errors, coverage_target)
        actual_coverage = np.mean(errors <= interval_width)
        results['overall'][f'coverage_{int(coverage_target*100)}'] = {
            'target': coverage_target,
            'actual': float(actual_coverage),
            'interval_width': float(interval_width),
            'calibrated': abs(actual_coverage - coverage_target) < 0.05
        }
    
    # Coverage by regime
    for regime_val, regime_name in [(0, 'normal'), (1, 'elevated'), (2, 'spike')]:
        mask = regime_labels == regime_val
        if mask.sum() < 50:
            continue
        
        regime_errors = errors[mask]
        regime_y = y.values[mask]
        regime_pred = y_pred[mask]
        
        # Standard conformal coverage
        in_interval = regime_errors <= conformal_quantile
        regime_coverage = np.mean(in_interval)
        
        # Adaptive coverage (if time calibration available)
        if time_calibration and uncertainty_scaler:
            hours = X.index.hour[mask] if hasattr(X.index, 'hour') else np.full(mask.sum(), 12)
            adaptive_coverages = []
            for i, (h, err) in enumerate(zip(hours, regime_errors)):
                period = 'afternoon' if 12 <= h < 18 else ('morning' if 6 <= h < 12 else ('evening' if 18 <= h < 24 else 'night'))
                mult = time_calibration.get(period, {}).get('multiplier', 1.0)
                adapted_interval = conformal_quantile * mult
                adaptive_coverages.append(err <= adapted_interval)
            adaptive_coverage = np.mean(adaptive_coverages)
        else:
            adaptive_coverage = regime_coverage
        
        results['by_regime'][regime_name] = {
            'n_samples': int(mask.sum()),
            'mae': float(np.mean(regime_errors)),
            'std': float(np.std(regime_errors)),
            'conformal_coverage': float(regime_coverage),
            'adaptive_coverage': float(adaptive_coverage),
            'target_coverage': 0.8,
            'calibrated': abs(regime_coverage - 0.8) < 0.1
        }
        
        if regime_coverage < 0.7:
            results['warnings'].append(f"{regime_name} regime under-covered: {regime_coverage:.1%} (target 80%)")
    
    # Coverage by time period
    hours = X.index.hour if hasattr(X.index, 'hour') else pd.Series(12, index=X.index)
    time_periods = {
        'night': (0, 6),
        'morning': (6, 12),
        'afternoon': (12, 18),
        'evening': (18, 24)
    }
    
    for period_name, (start, end) in time_periods.items():
        mask = (hours >= start) & (hours < end)
        if mask.sum() < 50:
            continue
        
        period_errors = errors[mask]
        
        # Standard conformal coverage
        in_interval = period_errors <= conformal_quantile
        period_coverage = np.mean(in_interval)
        
        # Adaptive coverage
        if time_calibration and period_name in time_calibration:
            mult = time_calibration[period_name].get('multiplier', 1.0)
            adapted_interval = conformal_quantile * mult
            adaptive_coverage = np.mean(period_errors <= adapted_interval)
        else:
            adaptive_coverage = period_coverage
        
        results['by_time'][period_name] = {
            'n_samples': int(mask.sum()),
            'mae': float(np.mean(period_errors)),
            'conformal_coverage': float(period_coverage),
            'adaptive_coverage': float(adaptive_coverage),
            'calibrated': abs(adaptive_coverage - 0.8) < 0.1
        }
        
        if period_coverage < 0.7:
            results['warnings'].append(f"{period_name} period under-covered: {period_coverage:.1%} (target 80%)")
    
    return results

for horizon in ['1h', '4h']:
    if horizon not in trained_models:
        continue
    if horizon not in conformal_residuals:
        continue
    
    print(f"\n{'='*50}")
    print(f"{horizon} Calibration Verification")
    print(f"{'='*50}")
    
    data = trained_models[horizon]
    model = data['model']
    scaler = data['scaler']
    features = data['features']
    conformal = conformal_residuals[horizon]
    time_cal = time_period_calibration.get(horizon, {})
    unc_scaler = uncertainty_scalers.get(horizon, None)
    
    if not HAS_HOLDOUT:
        print("  ⚠️ No holdout data for calibration verification")
        continue
    
    # Get holdout data
    X_test = df_holdout[features]
    y_test = df_holdout[f'target_{horizon}']
    mask = y_test.notna()
    X_test = X_test[mask]
    y_test = y_test[mask]
    
    # Create regime labels
    regime_test = pd.Series(0, index=X_test.index)
    if 'gas_zscore_1h' in df_holdout.columns:
        regime_test[df_holdout.loc[X_test.index, 'gas_zscore_1h'] > 1] = 1
    if 'is_spike' in df_holdout.columns:
        regime_test[df_holdout.loc[X_test.index, 'is_spike'] == 1] = 2
    
    # Verify calibration
    cal_results = verify_calibration_by_regime(
        X_test, y_test, model, scaler,
        conformal['quantile'], regime_test.values,
        time_cal, unc_scaler
    )
    
    # Print results
    print("\n  Overall Calibration:")
    for level, metrics in cal_results['overall'].items():
        status = "✓" if metrics['calibrated'] else "⚠"
        print(f"    {status} {int(metrics['target']*100)}% target: actual={metrics['actual']:.1%}, width={metrics['interval_width']:.4f}")
    
    print("\n  Calibration by Regime:")
    for regime_name, metrics in cal_results['by_regime'].items():
        status = "✓" if metrics['calibrated'] else "⚠"
        print(f"    {status} {regime_name}: conformal={metrics['conformal_coverage']:.1%}, adaptive={metrics['adaptive_coverage']:.1%} ({metrics['n_samples']} samples)")
    
    print("\n  Calibration by Time Period:")
    for period_name, metrics in cal_results['by_time'].items():
        status = "✓" if metrics['calibrated'] else "⚠"
        print(f"    {status} {period_name}: conformal={metrics['conformal_coverage']:.1%}, adaptive={metrics['adaptive_coverage']:.1%}")
    
    if cal_results['warnings']:
        print("\n  ⚠️ Calibration Warnings:")
        for warning in cal_results['warnings']:
            print(f"    - {warning}")
    
    # Store results
    regime_calibration[horizon] = cal_results
    
    # Update trained_models with detailed calibration
    if 'calibration' not in trained_models[horizon]:
        trained_models[horizon]['calibration'] = {}
    trained_models[horizon]['calibration']['regime_breakdown'] = cal_results['by_regime']
    trained_models[horizon]['calibration']['time_breakdown'] = cal_results['by_time']
    trained_models[horizon]['calibration']['warnings'] = cal_results['warnings']

# Summary
print(f"\n{'='*60}")
print("CALIBRATION SUMMARY")
print("="*60)

all_calibrated = True
for horizon, cal in regime_calibration.items():
    n_warnings = len(cal['warnings'])
    if n_warnings == 0:
        print(f"  ✓ {horizon}: All regimes and time periods well-calibrated")
    else:
        all_calibrated = False
        print(f"  ⚠ {horizon}: {n_warnings} calibration warning(s)")

if all_calibrated:
    print("\n✓ Prediction intervals are well-calibrated across all conditions")
else:
    print("\n⚠️ Some conditions show poor calibration - consider regime-specific intervals")

print(f"\n✓ Multi-regime calibration verification complete")

In [ ]:
# Direction Prediction - IMPROVED
# Changes: Binary up/down, class weights, holdout evaluation, adaptive threshold
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.utils.class_weight import compute_class_weight

print("\n" + "="*60)
print("TRAINING DIRECTION MODELS (IMPROVED)")
print("="*60)

direction_models = {}

# Configuration
USE_BINARY = True  # Binary (up/down) vs 3-class (down/stable/up)
DIRECTION_THRESHOLD = 0.01  # 1% threshold for direction change

def create_binary_direction(target, current, threshold=0.01):
    """Create binary direction labels: 1=up, 0=down/stable"""
    pct_change = (target - current) / (current + 1e-8)
    return (pct_change > threshold).astype(int)

def create_ternary_direction(target, current, threshold=0.02):
    """Create 3-class direction labels"""
    pct_change = (target - current) / (current + 1e-8)
    direction = pd.Series('stable', index=target.index)
    direction[pct_change > threshold] = 'up'
    direction[pct_change < -threshold] = 'down'
    return direction

for horizon in ['1h', '4h']:
    print(f"\n{'='*50}")
    print(f"{horizon.upper()} DIRECTION MODEL")
    print(f"{'='*50}")
    
    # Get features and targets
    X_h = X_1h if horizon == '1h' else X_4h
    features = numeric_features_1h if horizon == '1h' else numeric_features_4h
    
    # Get raw target for direction calculation
    if 'target_1h_raw' in df_train_val.columns:
        target_raw = df_train_val[f'target_{horizon}_raw']
    else:
        target_raw = df_train_val[f'target_{horizon}']
    
    current = df_train_val['gas']
    
    # Create direction labels
    if USE_BINARY:
        y_dir = create_binary_direction(target_raw, current, DIRECTION_THRESHOLD)
        print(f"Binary classification (threshold: {DIRECTION_THRESHOLD*100}%)")
    else:
        y_dir = create_ternary_direction(target_raw, current)
        print(f"3-class classification (threshold: {DIRECTION_THRESHOLD*100}%)")
    
    mask = y_dir.notna() & target_raw.notna()
    X_d = X_h[mask]
    y_d = y_dir[mask]
    
    if len(X_d) < 1000:
        print(f"  ⚠️ Insufficient data, skipping")
        continue
    
    # Class distribution
    class_counts = y_d.value_counts()
    print(f"Class distribution: {dict(class_counts)}")
    
    # Compute class weights
    classes = np.unique(y_d)
    weights = compute_class_weight('balanced', classes=classes, y=y_d)
    class_weight_dict = dict(zip(classes, weights))
    print(f"Class weights: {class_weight_dict}")
    
    # Split - use holdout if available
    if HAS_HOLDOUT:
        X_train, X_test = X_d, df_holdout[features]
        y_train = y_d
        
        # Create holdout labels
        if 'target_1h_raw' in df_holdout.columns:
            holdout_target = df_holdout[f'target_{horizon}_raw']
        else:
            holdout_target = df_holdout[f'target_{horizon}']
        holdout_current = df_holdout['gas']
        
        if USE_BINARY:
            y_test = create_binary_direction(holdout_target, holdout_current, DIRECTION_THRESHOLD)
        else:
            y_test = create_ternary_direction(holdout_target, holdout_current)
        
        test_mask = y_test.notna() & holdout_target.notna()
        X_test = X_test[test_mask]
        y_test = y_test[test_mask]
        print(f"Using holdout for evaluation ({len(y_test)} samples)")
    else:
        split_idx = int(len(X_d) * 0.8)
        X_train, X_test = X_d.iloc[:split_idx], X_d.iloc[split_idx:]
        y_train, y_test = y_d.iloc[:split_idx], y_d.iloc[split_idx:]
    
    scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Try multiple classifiers
    classifiers = [
        ('LogReg', LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)),
        ('RF', RandomForestClassifier(n_estimators=30, max_depth=4, class_weight='balanced', random_state=42, n_jobs=-1)),
        ('GBM', GradientBoostingClassifier(n_estimators=30, max_depth=3, learning_rate=0.1, random_state=42)),
    ]
    
    best_clf = None
    best_acc = 0
    best_name = None
    
    for name, clf in classifiers:
        try:
            clf.fit(X_train_scaled, y_train)
            y_pred = clf.predict(X_test_scaled)
            acc = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred, average='weighted')
            print(f"  {name}: Acc={acc:.1%}, F1={f1:.3f}")
            
            if acc > best_acc:
                best_acc = acc
                best_clf = clf
                best_name = name
                best_f1 = f1
        except Exception as e:
            print(f"  {name}: Failed - {e}")
    
    if best_clf is None:
        print(f"  ⚠️ All classifiers failed")
        continue
    
    # Baseline: always predict majority class
    majority_class = y_train.mode()[0]
    baseline_acc = (y_test == majority_class).mean()
    improvement = (best_acc - baseline_acc) / baseline_acc * 100
    
    print(f"\n  >>> Best: {best_name} (Acc: {best_acc:.1%}, vs baseline {baseline_acc:.1%}: {improvement:+.1f}%)")
    
    direction_models[horizon] = {
        'model': best_clf,
        'scaler': scaler,
        'accuracy': float(best_acc),
        'f1_score': float(best_f1),
        'baseline_accuracy': float(baseline_acc),
        'improvement_vs_baseline': float(improvement),
        'model_name': best_name,
        'is_binary': USE_BINARY,
        'threshold': DIRECTION_THRESHOLD
    }

# Summary
print(f"\n{'='*60}")
print("DIRECTION MODEL SUMMARY")
print(f"{'='*60}")
for horizon, data in direction_models.items():
    imp = data['improvement_vs_baseline']
    status = "✓" if imp > 5 else "⚠" if imp > 0 else "✗"
    print(f"{status} {horizon}: {data['model_name']} | Acc: {data['accuracy']:.1%} | vs baseline: {imp:+.1f}%")


In [ ]:
# REGIME DETECTION
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

print("\n" + "="*60)
print("TRAINING REGIME DETECTION MODEL")
print("="*60)

# Create regime labels from gas statistics (instead of volatility_regime)
# 0 = Normal, 1 = Elevated, 2 = Spike
if 'gas_zscore_1h' in df_train_val.columns and 'is_spike' in df_train_val.columns:
    # Create regime from z-score: low (<-0.5), normal (-0.5 to 1), elevated (1 to 2), spike (>2)
    zscore = df_train_val['gas_zscore_1h']
    is_spike = df_train_val['is_spike']
    
    regime_labels = pd.Series(0, index=df_train_val.index)  # Default: Normal
    regime_labels[zscore > 1] = 1  # Elevated
    regime_labels[is_spike == 1] = 2  # Spike
    
    X_r = X_4h.copy()
    y_r = regime_labels
    
    if len(X_r) < 500:
        print("⚠️ Insufficient data for regime detection")
        regime_clf = None
        regime_scaler = None
        regime_accuracy = 0
    else:
        # Train/test split
        split_idx = int(len(X_r) * 0.8)
        X_train, X_test = X_r.iloc[:split_idx], X_r.iloc[split_idx:]
        y_train, y_test = y_r.iloc[:split_idx], y_r.iloc[split_idx:]
        
        regime_scaler = RobustScaler()
        X_train_scaled = regime_scaler.fit_transform(X_train)
        X_test_scaled = regime_scaler.transform(X_test)
        
        # Train classifier (simple, reduced complexity)
        regime_clf = RandomForestClassifier(
            n_estimators=30, max_depth=4,
            min_samples_leaf=20,
            random_state=42, n_jobs=-1
        )
        regime_clf.fit(X_train_scaled, y_train)
        
        # Evaluate
        y_pred = regime_clf.predict(X_test_scaled)
        regime_accuracy = accuracy_score(y_test, y_pred)
        
        print(f"Regime classes: Normal (0), Elevated (1), Spike (2)")
        print(f"Class distribution: {dict(y_r.value_counts().sort_index())}")
        print(f"Accuracy: {regime_accuracy:.1%}")
        
        if regime_accuracy > 0.95:
            print("⚠️ Warning: Very high accuracy may indicate class imbalance or overfitting")
else:
    regime_clf = None
    regime_scaler = None
    regime_accuracy = 0
    print("⚠️ Missing gas_zscore_1h or is_spike, skipping regime detection")


In [ ]:
# Train Spike Detectors
from sklearn.ensemble import GradientBoostingClassifier

print("\n" + "="*60)
print("TRAINING SPIKE DETECTORS")
print("="*60)

spike_models = {}

for horizon, X_h, y_target in [('1h', X_1h, y_1h), ('4h', X_4h, y_4h)]:
    print(f"\n{horizon} spike detector...")
    
    # Create spike labels (>2 std from mean is a spike)
    mask = y_target.notna()
    X_s = X_h[mask]
    y_s = y_target[mask]
    current = current_gas[mask]
    
    # Define spike threshold
    price_change = y_s - current
    threshold = price_change.std() * 2
    spike_labels = (price_change > threshold).astype(int)
    
    spike_rate = spike_labels.mean()
    print(f"  Spike rate: {spike_rate:.1%}")
    
    if spike_rate < 0.01 or spike_rate > 0.5:
        print(f"  ⚠️ Unusual spike rate, skipping")
        continue
    
    if len(X_s) < 1000:
        print(f"  ⚠️ Insufficient data, skipping")
        continue
    
    # Train/test split
    split_idx = int(len(X_s) * 0.8)
    X_train, X_test = X_s.iloc[:split_idx], X_s.iloc[split_idx:]
    y_train, y_test = spike_labels.iloc[:split_idx], spike_labels.iloc[split_idx:]
    
    scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Train with class weights
    clf = GradientBoostingClassifier(
        n_estimators=50, max_depth=4,
        learning_rate=0.1, random_state=42
    )
    clf.fit(X_train_scaled, y_train)
    
    # Evaluate
    y_pred = clf.predict(X_test_scaled)
    acc = accuracy_score(y_test, y_pred)
    
    spike_models[horizon] = (clf, scaler)
    print(f"  Accuracy: {acc:.1%}")

# Copy 4h to 24h if available
if '4h' in spike_models:
    spike_models['24h'] = spike_models['4h']
    print("\n24h: Using 4h spike detector (fallback)")

print(f"\n✓ Spike detectors trained for: {list(spike_models.keys())}")

In [ ]:
# SPIKE FORECASTING - IMPROVED with Multiple Definitions & Volatility Featuresfrom sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifierfrom sklearn.linear_model import LogisticRegressionfrom sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curveimport warningswarnings.filterwarnings('ignore')print("\n" + "="*60)print("SPIKE FORECASTING (IMPROVED)")print("="*60)# Check for SMOTEtry:    from imblearn.over_sampling import SMOTE    HAS_SMOTE = Trueexcept ImportError:    HAS_SMOTE = Falsespike_forecast_models = {}def create_volatility_features(df):    """    Create comprehensive volatility and regime-transition features.    Focus on patterns that precede spikes, not just rate-of-change.    """    rph = 120  # rows per hour    features = {}        # === VOLATILITY CLUSTERING (GARCH-like) ===    # Squared returns as volatility proxy    returns = df['gas'].pct_change()    features['volatility_5min'] = returns.rolling(10).std()    features['volatility_15min'] = returns.rolling(30).std()    features['volatility_1h'] = returns.rolling(rph).std()        # Volatility of volatility (regime change indicator)    features['vol_of_vol'] = features['volatility_15min'].rolling(rph).std()        # Volatility ratio (short/long) - high ratio = volatility increasing    features['vol_ratio'] = features['volatility_5min'] / (features['volatility_1h'] + 1e-8)        # === REGIME TRANSITION FEATURES ===    # Moving average crossovers    ma_5min = df['gas'].rolling(10).mean()    ma_30min = df['gas'].rolling(60).mean()    ma_1h = df['gas'].rolling(rph).mean()    features['ma_cross_5_30'] = (ma_5min - ma_30min) / (ma_30min + 1e-8)    features['ma_cross_30_60'] = (ma_30min - ma_1h) / (ma_1h + 1e-8)        # Distance from recent low (potential for mean reversion spike)    rolling_low_1h = df['gas'].rolling(rph).min()    features['dist_from_low'] = (df['gas'] - rolling_low_1h) / (rolling_low_1h + 1e-8)        # Distance from recent high    rolling_high_1h = df['gas'].rolling(rph).max()    features['dist_from_high'] = (rolling_high_1h - df['gas']) / (df['gas'] + 1e-8)        # Range as % of price (consolidation vs expansion)    features['range_pct'] = (rolling_high_1h - rolling_low_1h) / (df['gas'] + 1e-8)        # === ABSOLUTE LEVEL FEATURES ===    # Current gas relative to historical percentiles    rolling_median_4h = df['gas'].rolling(4 * rph).median()    features['above_median_4h'] = (df['gas'] > rolling_median_4h).astype(float)    features['pct_above_median'] = (df['gas'] - rolling_median_4h) / (rolling_median_4h + 1e-8)        # Absolute gas level buckets (spikes often start from low base)    features['gas_level'] = df['gas']    features['is_low_gas'] = (df['gas'] < df['gas'].rolling(4 * rph).quantile(0.25)).astype(float)    features['is_high_gas'] = (df['gas'] > df['gas'].rolling(4 * rph).quantile(0.75)).astype(float)        # === MOMENTUM FEATURES ===    features['momentum_15min'] = df['gas'].diff(30)    features['momentum_1h'] = df['gas'].diff(rph)    features['momentum_accel'] = features['momentum_15min'].diff(30)        # Rate of change    features['roc_5min'] = df['gas'].pct_change(10)    features['roc_15min'] = df['gas'].pct_change(30)    features['roc_1h'] = df['gas'].pct_change(rph)        # === TIME FEATURES ===    if hasattr(df.index, 'hour'):        features['hour'] = df.index.hour        features['hour_sin'] = np.sin(2 * np.pi * df.index.hour / 24)        features['hour_cos'] = np.cos(2 * np.pi * df.index.hour / 24)        # High activity hours (when spikes are more likely)        features['is_active_hours'] = ((df.index.hour >= 13) & (df.index.hour <= 21)).astype(float)        return pd.DataFrame(features, index=df.index)def create_spike_target_multi(df, horizon_hours, definitions):    """    Create spike target using multiple definitions.    Returns dict of targets for each definition.    """    rph = 120    horizon_periods = horizon_hours * rph    targets = {}        # Calculate forward max    future_max = df['gas'].shift(-1).rolling(horizon_periods, min_periods=1).max()    future_max = future_max.shift(-horizon_periods + 1)    current_gas = df['gas']    pct_change = (future_max - current_gas) / (current_gas + 1e-8)    abs_change = future_max - current_gas        for defn in definitions:        name = defn['name']        if defn['type'] == 'pct':            targets[name] = (pct_change > defn['threshold']).astype(int)        elif defn['type'] == 'abs':            targets[name] = (abs_change > defn['threshold']).astype(int)        elif defn['type'] == 'combined':            targets[name] = ((pct_change > defn['pct_threshold']) |                             (abs_change > defn['abs_threshold'])).astype(int)        elif defn['type'] == 'percentile':            # Spike = future max is in top X percentile            threshold = df['gas'].rolling(4 * rph, min_periods=rph).quantile(defn['percentile'])            targets[name] = (future_max > threshold).astype(int)                targets[name] = targets[name].fillna(0).astype(int)        return targetsdef find_optimal_threshold(y_true, y_prob):    """Find threshold that maximizes F1"""    precision, recall, thresholds = precision_recall_curve(y_true, y_prob)    f1_scores = 2 * (precision[:-1] * recall[:-1]) / (precision[:-1] + recall[:-1] + 1e-8)    if len(f1_scores) == 0:        return 0.5, 0.0    best_idx = np.argmax(f1_scores)    return thresholds[best_idx], f1_scores[best_idx]# === CREATE FEATURES ===print("\nCreating volatility features...")vol_features_train = create_volatility_features(df_train_val)if HAS_HOLDOUT:    vol_features_holdout = create_volatility_features(df_holdout)# Add to dataframesfor col in vol_features_train.columns:    if col not in df_train_val.columns:        df_train_val[col] = vol_features_train[col]        if HAS_HOLDOUT:            df_holdout[col] = vol_features_holdout[col]# Select features for spike prediction# IMPROVED: Horizon-specific feature sets# Shorter horizons need shorter lookback windowsSPIKE_FEATURES_SHORT = [  # For 1h/2h horizons    'hour_sin', 'hour_cos', 'is_active_hours',    'volatility_5min', 'volatility_15min',  # Short-term vol only    'vol_ratio',    'ma_cross_5_30',    'dist_from_low', 'dist_from_high',    'is_low_gas', 'is_high_gas',    'momentum_15min', 'momentum_accel',  # Fast momentum    'roc_5min', 'roc_15min'  # Fast ROC]SPIKE_FEATURES_LONG = [  # For 4h+ horizons    'hour_sin', 'hour_cos', 'is_active_hours',    'volatility_5min', 'volatility_15min', 'volatility_1h',    'vol_of_vol', 'vol_ratio',    'ma_cross_5_30', 'ma_cross_30_60',    'dist_from_low', 'dist_from_high', 'range_pct',    'is_low_gas', 'is_high_gas', 'pct_above_median',    'momentum_15min', 'momentum_1h', 'momentum_accel',    'roc_5min', 'roc_15min', 'roc_1h']# Select features based on horizondef get_spike_features_for_horizon(horizon_hours):    if horizon_hours <= 2:        return SPIKE_FEATURES_SHORT    else:        return SPIKE_FEATURES_LONGspike_features = SPIKE_FEATURES_LONG  # Default for compatibilityavailable_features = [f for f in spike_features if f in df_train_val.columns]print(f"Using {len(available_features)} features for spike forecasting")# === SPIKE DEFINITIONS TO TRY ===SPIKE_DEFINITIONS = [    {'name': 'pct_15', 'type': 'pct', 'threshold': 0.15},    {'name': 'pct_25', 'type': 'pct', 'threshold': 0.25},    {'name': 'pct_50', 'type': 'pct', 'threshold': 0.50},    {'name': 'abs_5gwei', 'type': 'abs', 'threshold': 5.0},    {'name': 'abs_10gwei', 'type': 'abs', 'threshold': 10.0},    {'name': 'combined', 'type': 'combined', 'pct_threshold': 0.20, 'abs_threshold': 5.0},    {'name': 'top10pct', 'type': 'percentile', 'percentile': 0.90},]# === TRAIN SPIKE FORECASTERS ===for horizon_name, horizon_hours in [('1h', 1), ('2h', 2), ('4h', 4)]:  # Ordered by horizon    # Get horizon-specific features    horizon_spike_features = get_spike_features_for_horizon(horizon_hours)    available_features = [f for f in horizon_spike_features if f in df_train_val.columns]    print(f"  Using {len(available_features)} features for {horizon_name} spike forecast")    print(f"\n{'='*50}")    print(f"{horizon_name} Spike Forecast Model")    print(f"{'='*50}")        best_model_result = None    best_auc = 0        # Create all spike targets    spike_targets_train = create_spike_target_multi(df_train_val, horizon_hours, SPIKE_DEFINITIONS)    if HAS_HOLDOUT:        spike_targets_holdout = create_spike_target_multi(df_holdout, horizon_hours, SPIKE_DEFINITIONS)        for defn in SPIKE_DEFINITIONS:        defn_name = defn['name']        print(f"\n  Testing {defn_name}...")                y_sf = spike_targets_train[defn_name]        X_sf = df_train_val[available_features].copy()  # Uses horizon-specific features                # Remove NaN        mask = y_sf.notna() & X_sf.notna().all(axis=1)        X_sf = X_sf[mask]        y_sf = y_sf[mask]                if len(X_sf) < 1000:            print(f"    ⚠️ Insufficient data ({len(X_sf)} samples)")            continue                spike_rate = y_sf.mean()        print(f"    Spike rate: {spike_rate:.1%} ({y_sf.sum():.0f} spikes)")                # Skip extreme imbalance        if spike_rate > 0.7 or spike_rate < 0.01:            print(f"    ⚠️ Class imbalance too extreme")            continue                # Prepare test set        if HAS_HOLDOUT:            X_train = X_sf            y_train = y_sf                        y_test = spike_targets_holdout[defn_name]            X_test = df_holdout[available_features].copy()                        mask_test = y_test.notna() & X_test.notna().all(axis=1)            X_test = X_test[mask_test]            y_test = y_test[mask_test]        else:            split_idx = int(len(X_sf) * 0.8)            X_train, X_test = X_sf.iloc[:split_idx], X_sf.iloc[split_idx:]            y_train, y_test = y_sf.iloc[:split_idx], y_sf.iloc[split_idx:]                if len(X_test) < 100 or y_test.sum() < 5:            print(f"    ⚠️ Insufficient test data")            continue                scaler = RobustScaler()        X_train_scaled = scaler.fit_transform(X_train)        X_test_scaled = scaler.transform(X_test)                # Handle class imbalance        use_smote = HAS_SMOTE and y_train.sum() >= 10 and (y_train == 0).sum() >= 10                if use_smote:            try:                k_neighbors = min(5, int(min(y_train.sum(), (y_train == 0).sum())) - 1)                k_neighbors = max(1, k_neighbors)                smote = SMOTE(random_state=42, k_neighbors=k_neighbors)                X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)            except:                X_train_resampled, y_train_resampled = X_train_scaled, y_train                use_smote = False        else:            X_train_resampled, y_train_resampled = X_train_scaled, y_train                # Try classifiers        classifiers = [            ('GBM', GradientBoostingClassifier(n_estimators=100, max_depth=4, learning_rate=0.1,                                                min_samples_leaf=20, subsample=0.8, random_state=42)),            ('RF', RandomForestClassifier(n_estimators=100, max_depth=5, min_samples_leaf=20,                                          class_weight='balanced', random_state=42, n_jobs=-1)),            ('LogReg', LogisticRegression(C=0.1, class_weight='balanced', max_iter=1000, random_state=42))        ]                for clf_name, clf in classifiers:            try:                if use_smote:                    clf.fit(X_train_resampled, y_train_resampled)                else:                    clf.fit(X_train_scaled, y_train)                                y_prob = clf.predict_proba(X_test_scaled)[:, 1]                                # Calculate AUC                try:                    auc = roc_auc_score(y_test, y_prob)                except:                    auc = 0.5                                # Only consider if AUC > 0.55 (better than random)                if auc > best_auc and auc > 0.55:                    optimal_threshold, optimal_f1 = find_optimal_threshold(y_test, y_prob)                    y_pred = (y_prob >= optimal_threshold).astype(int)                                        precision = precision_score(y_test, y_pred, zero_division=0)                    recall = recall_score(y_test, y_pred, zero_division=0)                    f1 = f1_score(y_test, y_pred, zero_division=0)                                        # Must have reasonable precision (>50%) to be useful                    if precision >= 0.5:                        best_auc = auc                        best_model_result = {                            'model': clf,                            'scaler': scaler,                            'spike_definition': defn,                            'classifier': clf_name,                            'optimal_threshold': float(optimal_threshold),                            'precision': float(precision),                            'recall': float(recall),                            'f1_score': float(f1),                            'auc': float(auc),                            'spike_rate_train': float(spike_rate),                            'spike_rate_test': float(y_test.mean()),                            'features': available_features,                            'horizon': horizon_name                        }                        print(f"    {clf_name}: AUC={auc:.3f}, P={precision:.1%}, R={recall:.1%}, F1={f1:.3f} ✓")                    else:                        print(f"    {clf_name}: AUC={auc:.3f}, P={precision:.1%} (too low)")                                    except Exception as e:                pass        # Store results    if best_model_result:        spike_forecast_models[horizon_name] = best_model_result        print(f"\n  ✓ Best: {best_model_result['classifier']} with {best_model_result['spike_definition']['name']}")        print(f"    AUC={best_model_result['auc']:.3f}, P={best_model_result['precision']:.1%}, R={best_model_result['recall']:.1%}")    else:        print(f"\n  ⚠️ No viable {horizon_name} spike model (need AUC>0.55 and P>50%)")        spike_forecast_models[horizon_name] = {            'model': None,            'scaler': None,            'precision': 0.0,            'recall': 0.0,            'f1_score': 0.0,            'auc': 0.5,            'has_model': False,            'note': 'No model met quality thresholds'        }print(f"\n{'='*60}")print("SPIKE FORECAST SUMMARY")print("="*60)for h, data in spike_forecast_models.items():    if data.get('model') is not None:        print(f"  {h}: AUC={data['auc']:.3f}, P={data['precision']:.1%}, R={data['recall']:.1%}")        print(f"      Definition: {data['spike_definition']['name']}")    else:        print(f"  {h}: No viable model")

In [ ]:
# DYNAMIC ENSEMBLE WEIGHTING + CONFIDENCE SCORINGprint("\n" + "="*60)print("DYNAMIC ENSEMBLE WEIGHTING + CONFIDENCE SCORING")print("="*60)def calculate_dynamic_weights(global_errors, regime_errors, decay=0.9):    """Calculate dynamic weights based on exponentially weighted recent errors"""    if len(global_errors) == 0 or len(regime_errors) == 0:        return 0.5, 0.5        n = len(global_errors)    exp_weights = np.array([decay ** (n - i - 1) for i in range(n)])    exp_weights = exp_weights / exp_weights.sum()        global_weighted_error = np.sum(global_errors * exp_weights)    regime_weighted_error = np.sum(regime_errors * exp_weights)        total_inv_error = 1/(global_weighted_error + 1e-8) + 1/(regime_weighted_error + 1e-8)    global_weight = (1/(global_weighted_error + 1e-8)) / total_inv_error    regime_weight = (1/(regime_weighted_error + 1e-8)) / total_inv_error        return global_weight, regime_weightdef compute_prediction_confidence(predictions, base_interval, volatility_ratio=1.0):    """    Compute confidence score based on model agreement and volatility.    Returns confidence (0-1) and suggested interval multiplier.    """    if len(predictions) <= 1:        return 1.0, 1.0  # High confidence, no multiplier        # Model disagreement as spread    pred_std = np.std(predictions)    pred_mean = np.mean(predictions)    pred_cv = pred_std / (np.abs(pred_mean) + 1e-8)  # Coefficient of variation        # Normalize disagreement (higher = less confident)    # pred_cv > 0.5 means models disagree by more than 50%    disagreement = min(pred_cv / 0.5, 1.0)        # Confidence = 1 - disagreement, adjusted for volatility    confidence = (1 - disagreement) / (1 + volatility_ratio * 0.5)    confidence = max(0.2, min(1.0, confidence))        # Interval multiplier: widen when confidence is low    # confidence=1.0 -> multiplier=1.0, confidence=0.5 -> multiplier=1.5    multiplier = 1.0 + (1.0 - confidence)        return confidence, multiplierclass ConfidenceScoringPredictor:    """Predictor that outputs confidence scores alongside predictions."""        def __init__(self, main_model, main_scaler, features,                  quantile_models=None, quantile_scaler=None,                 regime_models=None, base_interval=0.3):        self.main_model = main_model        self.main_scaler = main_scaler        self.features = features        self.quantile_models = quantile_models        self.quantile_scaler = quantile_scaler        self.regime_models = regime_models        self.base_interval = base_interval        self.recent_errors = []        self.max_history = 100        def predict_with_confidence(self, X, hour=None, regime=None, volatility_ratio=1.0):        """        ENHANCED: Make prediction with comprehensive confidence scoring.        Returns dict with prediction, confidence, intervals, and diagnostics.        """        # Prepare input        if hasattr(X, 'columns'):            X_subset = X[self.features] if all(f in X.columns for f in self.features) else X        else:            X_subset = X        # Get main prediction        X_scaled = self.main_scaler.transform(X_subset)        main_pred = self.main_model.predict(X_scaled)        # Collect predictions from different models        all_predictions = [main_pred]        model_sources = ['main']        # Add quantile median if available        if self.quantile_models and self.quantile_scaler:            try:                X_q_scaled = self.quantile_scaler.transform(X_subset)                if 0.5 in self.quantile_models:                    q_pred = self.quantile_models[0.5].predict(X_q_scaled)                    all_predictions.append(q_pred)                    model_sources.append('quantile_median')            except:                pass        # Add regime model prediction if available        if regime is not None and self.regime_models and regime in self.regime_models:            try:                regime_data = self.regime_models[regime]                regime_features = regime_data.get('features', self.features)                if hasattr(X, 'columns'):                    available = [f for f in regime_features if f in X.columns]                    if len(available) == len(regime_features):                        X_regime = X[regime_features]                        X_regime_scaled = regime_data['scaler'].transform(X_regime)                        regime_pred = regime_data['model'].predict(X_regime_scaled)                        all_predictions.append(regime_pred)                        model_sources.append(f'regime_{regime}')            except:                pass        # Compute model disagreement        all_preds_array = np.array([p.flatten()[0] if hasattr(p, 'flatten') else p for p in all_predictions])        model_disagreement = float(np.std(all_preds_array)) if len(all_preds_array) > 1 else 0.0        pred_cv = model_disagreement / (np.abs(np.mean(all_preds_array)) + 1e-8)        # === ENHANCED CONFIDENCE CALCULATION ===        # 1. Model agreement component (0-1, higher = more agreement)        agreement_score = max(0, 1 - (pred_cv / 0.3))        # 2. Rolling performance component (based on recent hit rate)        performance_score = (self.hit_rate_80 + self.hit_rate_90) / 2        # 3. Volatility component (lower confidence in high volatility)        volatility_score = 1.0 / (1.0 + volatility_ratio * 0.5)        # 4. Regime component (lower confidence in spike/elevated regimes)        regime_score = 1.0        if regime is not None:            regime_scores = {0: 1.0, 1: 0.8, 2: 0.7}            regime_score = regime_scores.get(regime, 0.9)        # 5. Time component (lower confidence in volatile hours)        time_score = 1.0        if hour is not None:            if 12 <= hour < 18:                time_score = 0.85            elif 0 <= hour < 6:                time_score = 0.9        # Combine scores (weighted geometric mean)        weights = [0.35, 0.25, 0.15, 0.15, 0.10]        scores = [agreement_score, performance_score, volatility_score, regime_score, time_score]        confidence = np.prod([s ** w for s, w in zip(scores, weights)])        confidence = max(0.2, min(1.0, confidence))        # === ADAPTIVE INTERVAL CALCULATION ===        interval_multiplier = 1.0 + (1.0 - confidence) * 0.5        if hour is not None and 12 <= hour < 18:            interval_multiplier *= 1.15        if regime == 2:            interval_multiplier *= 1.2        interval_width = self.base_interval * interval_multiplier        if len(self.recent_errors) >= 10:            recent_mae = np.mean(self.recent_errors[-10:])            interval_width = max(interval_width, recent_mae * 1.5)        pred_value = main_pred.flatten()[0] if hasattr(main_pred, 'flatten') else float(main_pred)        return {            'prediction': pred_value,            'confidence': float(confidence),            'interval_low': pred_value - interval_width,            'interval_high': pred_value + interval_width,            'interval_width': float(interval_width),            'model_disagreement': float(model_disagreement),            'n_models': len(all_predictions),            'model_sources': model_sources,            'confidence_components': {                'agreement': float(agreement_score),                'performance': float(performance_score),                'volatility': float(volatility_score),                'regime': float(regime_score),                'time': float(time_score)            },            'rolling_hit_rate_80': float(self.hit_rate_80),            'rolling_mae': float(self.rolling_mae)        }    def update_with_actual(self, prediction, actual, interval_width=None):        """        ENHANCED: Update rolling metrics for adaptive confidence.        Tracks hit rates for 80% and 90% intervals.        """        error = abs(actual - prediction)        self.recent_errors.append(error)        self.recent_predictions.append(prediction)        self.recent_actuals.append(actual)        # Maintain history size        if len(self.recent_errors) > self.max_history:            self.recent_errors.pop(0)            self.recent_predictions.pop(0)            self.recent_actuals.pop(0)        # Update rolling MAE        if len(self.recent_errors) >= 10:            self.rolling_mae = np.mean(self.recent_errors[-min(50, len(self.recent_errors)):])        # Update hit rates if interval was provided        if interval_width is not None:            interval_80 = interval_width            interval_90 = interval_width * 1.3            # Calculate rolling hit rates            recent_n = min(50, len(self.recent_errors))            recent_errors = np.array(self.recent_errors[-recent_n:])            self.hit_rate_80 = np.mean(recent_errors <= interval_80)            self.hit_rate_90 = np.mean(recent_errors <= interval_90)def create_dynamic_ensemble_predictor(global_model, global_scaler, regime_models, features, decay=0.9):    """Create ensemble predictor with dynamic weighting and confidence scoring."""    recent_errors = {'global': [], 'regime': {}}        def predict(X, current_regime=None, actual_value=None):        nonlocal recent_errors                if hasattr(X, 'columns'):            X_global = X[features] if all(f in X.columns for f in features) else X            X_scaled = global_scaler.transform(X_global)        else:            X_scaled = global_scaler.transform(X)                global_pred = global_model.predict(X_scaled)                if current_regime is not None and regime_models and current_regime in regime_models:            regime_data = regime_models[current_regime]            regime_features = regime_data.get('features', features)                        try:                if hasattr(X, 'columns'):                    available_features = [f for f in regime_features if f in X.columns]                    if len(available_features) == len(regime_features):                        X_regime = X[regime_features]                        X_regime_scaled = regime_data['scaler'].transform(X_regime)                        regime_pred = regime_data['model'].predict(X_regime_scaled)                    else:                        return global_pred                else:                    expected_features = regime_data['scaler'].n_features_in_                    if X.shape[1] == expected_features:                        X_regime_scaled = regime_data['scaler'].transform(X)                        regime_pred = regime_data['model'].predict(X_regime_scaled)                    else:                        return global_pred            except Exception:                return global_pred                        if current_regime not in recent_errors['regime']:                recent_errors['regime'][current_regime] = []                        global_errors = np.array(recent_errors['global']) if recent_errors['global'] else np.array([1.0])            regime_errors = np.array(recent_errors['regime'].get(current_regime, [1.0]))                        g_weight, r_weight = calculate_dynamic_weights(global_errors, regime_errors, decay)            final_pred = g_weight * global_pred + r_weight * regime_pred                        if actual_value is not None:                g_error = abs(actual_value - global_pred)                r_error = abs(actual_value - regime_pred)                recent_errors['global'].append(float(g_error))                recent_errors['regime'][current_regime].append(float(r_error))                                if len(recent_errors['global']) > 100:                    recent_errors['global'].pop(0)                if len(recent_errors['regime'][current_regime]) > 100:                    recent_errors['regime'][current_regime].pop(0)                        return final_pred                return global_pred        return predict# Create confidence scoring predictors for each horizonconfidence_predictors = {}for horizon in ['1h', '4h']:    if horizon not in trained_models:        continue        data = trained_models[horizon]        # Get quantile models if available    q_models = None    q_scaler = None    if horizon in quantile_models:        q_models, q_scaler = quantile_models[horizon]        # Get regime models if available    r_models = None    if 'regime_specific_models' in dir() and regime_specific_models.get(horizon):        r_models = regime_specific_models[horizon]        # Get base interval from conformal calibration    base_interval = conformal_residuals.get(horizon, {}).get('quantile', 0.3)        # Create confidence predictor    conf_pred = ConfidenceScoringPredictor(        main_model=data['model'],        main_scaler=data['scaler'],        features=data['features'],        quantile_models=q_models,        quantile_scaler=q_scaler,        regime_models=r_models,        base_interval=base_interval    )        confidence_predictors[horizon] = conf_pred        # Store in trained_models for saving    trained_models[horizon]['confidence_predictor'] = conf_pred    trained_models[horizon]['has_confidence_scoring'] = True        print(f"\n{horizon}: Confidence scoring predictor created")    print(f"  Base interval: ±{base_interval:.4f}")    print(f"  Quantile models: {'Yes' if q_models else 'No'}")    print(f"  Regime models: {'Yes' if r_models else 'No'}")# Also create dynamic ensemble predictorsfor horizon in ['1h', '4h']:    if horizon not in trained_models:        continue        data = trained_models[horizon]    r_models = None    if 'regime_specific_models' in dir() and regime_specific_models.get(horizon):        r_models = regime_specific_models[horizon]        if r_models:        ensemble_pred = create_dynamic_ensemble_predictor(            data['model'], data['scaler'], r_models, data['features']        )        trained_models[horizon]['dynamic_ensemble_predict'] = ensemble_pred        trained_models[horizon]['has_dynamic_ensemble'] = True        print(f"  {horizon}: Dynamic ensemble predictor created")print(f"\n✓ Confidence scoring and dynamic ensemble setup complete")

In [ ]:
# ERROR ANALYSIS + AUTOMATIC BIAS CORRECTION (IMPROVED)
print("\n" + "="*60)
print("ERROR ANALYSIS + BIAS CORRECTION")
print("="*60)

error_analysis = {}
bias_correction_factors = {}

# === MODULE-LEVEL CLASS FOR PICKLING ===
class BiasCorrectedModel:
    """
    Model wrapper that automatically applies bias correction during inference.
    Defined at module level for pickling compatibility.
    """
    def __init__(self, model, scaler, bias_factors, features):
        self.model = model
        self.scaler = scaler
        self.bias_factors = bias_factors
        self.features = features
    
    def predict(self, X, hour=None, regime=None):
        """Predict with automatic bias correction"""
        if hasattr(X, 'columns'):
            X_subset = X[self.features]
        else:
            X_subset = X
        
        X_scaled = self.scaler.transform(X_subset)
        raw_pred = self.model.predict(X_scaled)
        
        # Apply bias correction
        correction = 0.0
        
        # Time-based correction (most specific)
        if hour is not None and 'by_time' in self.bias_factors:
            period = 'afternoon' if 12 <= hour < 18 else (
                'morning' if 6 <= hour < 12 else (
                'evening' if 18 <= hour < 24 else 'night'))
            
            if period in self.bias_factors['by_time']:
                tb = self.bias_factors['by_time'][period]
                if tb.get('should_apply', False):
                    correction = tb['correction']
        
        # Regime-based correction (fallback)
        elif regime is not None and 'by_regime' in self.bias_factors:
            regime_name = {0: 'normal', 1: 'elevated', 2: 'spike'}.get(regime)
            if regime_name in self.bias_factors['by_regime']:
                rb = self.bias_factors['by_regime'][regime_name]
                if rb.get('should_apply', False):
                    correction = rb['correction']
        
        # Overall correction (last resort)
        elif self.bias_factors.get('overall', {}).get('should_apply', False):
            correction = self.bias_factors['overall']['correction']
        
        return raw_pred + correction
    
    def predict_raw(self, X):
        """Predict without bias correction"""
        if hasattr(X, 'columns'):
            X_subset = X[self.features]
        else:
            X_subset = X
        X_scaled = self.scaler.transform(X_subset)
        return self.model.predict(X_scaled)
    
    def get_params(self, deep=True):
        return {}

def compute_bias_correction(y_true, y_pred, hours, regimes):
    """
    Compute bias correction factors that can be applied during inference.
    IMPROVED: Use median for robustness, lower threshold for high-error periods.
    """
    errors = y_true - y_pred  # Positive = under-prediction, Negative = over-prediction
    abs_errors = np.abs(errors)
    
    # Overall bias
    overall_bias = np.mean(errors)
    overall_median_bias = np.median(errors)
    recent_bias = np.mean(errors[-min(240, len(errors)):])  # Last 2 hours
    
    # Time-period specific bias
    time_bias = {}
    time_periods = {
        'night': (0, 6),
        'morning': (6, 12),
        'afternoon': (12, 18),
        'evening': (18, 24)
    }
    
    overall_mae = np.mean(abs_errors)
    
    for period_name, (start, end) in time_periods.items():
        mask = (hours >= start) & (hours < end)
        if mask.sum() >= 50:
            period_errors = errors[mask]
            period_abs_errors = abs_errors[mask]
            period_bias = np.mean(period_errors)
            period_median_bias = np.median(period_errors)
            period_mae = np.mean(period_abs_errors)
            
            # Use median bias for correction (more robust)
            correction = period_median_bias
            
            # Apply if bias > 2% (lowered from 3%) OR if this period has much higher MAE
            is_high_error_period = period_mae > overall_mae * 1.2
            should_apply = abs(period_bias) > 0.02 or (is_high_error_period and abs(period_bias) > 0.01)
            
            time_bias[period_name] = {
                'bias': float(period_bias),
                'median_bias': float(period_median_bias),
                'correction': float(correction),
                'mae': float(period_mae),
                'mae_ratio': float(period_mae / overall_mae) if overall_mae > 0 else 1.0,
                'n_samples': int(mask.sum()),
                'should_apply': should_apply,
                'is_high_error': is_high_error_period
            }
    
    # Regime-specific bias
    regime_bias = {}
    for regime_val, regime_name in [(0, 'normal'), (1, 'elevated'), (2, 'spike')]:
        mask = regimes == regime_val
        if mask.sum() >= 50:
            r_errors = errors[mask]
            r_bias = np.mean(r_errors)
            r_median_bias = np.median(r_errors)
            
            regime_bias[regime_name] = {
                'bias': float(r_bias),
                'median_bias': float(r_median_bias),
                'correction': float(r_median_bias),
                'n_samples': int(mask.sum()),
                'should_apply': abs(r_bias) > 0.02
            }
    
    return {
        'overall': {
            'bias': float(overall_bias),
            'median_bias': float(overall_median_bias),
            'recent_bias': float(recent_bias),
            'correction': float(overall_median_bias),
            'mae': float(overall_mae),
            'should_apply': abs(overall_median_bias) > 0.05
        },
        'by_time': time_bias,
        'by_regime': regime_bias
    }

def create_bias_corrected_predictor(model, scaler, bias_factors, features):
    """Create a predictor that automatically applies bias correction."""
    return BiasCorrectedModel(model, scaler, bias_factors, features)

for horizon in ['1h', '4h']:
    if horizon not in trained_models:
        continue
    
    print(f"\n{horizon} Error Analysis...")
    
    data = trained_models[horizon]
    model = data['model']
    scaler = data['scaler']
    features = data['features']
    
    if not HAS_HOLDOUT:
        print("  ⚠️ No holdout data")
        continue
    
    # Get predictions on holdout
    X_test = df_holdout[features]
    y_test = df_holdout[f'target_{horizon}']
    
    mask = y_test.notna()
    X_test = X_test[mask]
    y_test = y_test[mask]
    
    X_test_scaled = scaler.transform(X_test)
    y_pred = model.predict(X_test_scaled)
    
    errors = y_test.values - y_pred
    abs_errors = np.abs(errors)
    
    # Get hours and regimes
    hours = X_test.index.hour if hasattr(X_test.index, 'hour') else np.full(len(X_test), 12)
    
    regimes = np.zeros(len(X_test))
    if 'gas_zscore_1h' in df_holdout.columns:
        regimes[df_holdout.loc[X_test.index, 'gas_zscore_1h'] > 1] = 1
    if 'is_spike' in df_holdout.columns:
        regimes[df_holdout.loc[X_test.index, 'is_spike'] == 1] = 2
    
    # === COMPUTE BIAS CORRECTION FACTORS ===
    bias_factors = compute_bias_correction(y_test.values, y_pred, hours, regimes)
    bias_correction_factors[horizon] = bias_factors
    
    print(f"\n  Bias Analysis:")
    print(f"    Overall: mean={bias_factors['overall']['bias']:.4f}, median={bias_factors['overall']['median_bias']:.4f}")
    
    print(f"\n  Time-Period Bias:")
    for period, tb in bias_factors['by_time'].items():
        apply_flag = "✓ APPLY" if tb['should_apply'] else "  skip"
        high_err = " [HIGH ERROR]" if tb.get('is_high_error', False) else ""
        print(f"    {period}: bias={tb['bias']:+.4f}, MAE={tb['mae']:.4f} ({tb['mae_ratio']:.1f}x){high_err} {apply_flag}")
    
    print(f"\n  Regime Bias:")
    for regime_name, rb in bias_factors['by_regime'].items():
        apply_flag = "✓ APPLY" if rb['should_apply'] else "  skip"
        print(f"    {regime_name}: {rb['bias']:+.4f} ({rb['n_samples']} samples) {apply_flag}")
    
    # === TEST BIAS-CORRECTED PREDICTIONS ===
    print(f"\n  Bias Correction Test:")
    
    # Predictions without correction
    raw_mae = np.mean(abs_errors)
    
    # Predictions with time-based correction only
    corrected_preds_time = y_pred.copy()
    for i in range(len(corrected_preds_time)):
        h = hours[i]
        period = 'afternoon' if 12 <= h < 18 else (
            'morning' if 6 <= h < 12 else (
            'evening' if 18 <= h < 24 else 'night'))
        
        if period in bias_factors['by_time'] and bias_factors['by_time'][period]['should_apply']:
            corrected_preds_time[i] += bias_factors['by_time'][period]['correction']
    
    corrected_mae_time = mean_absolute_error(y_test, corrected_preds_time)
    improvement_time = (raw_mae - corrected_mae_time) / raw_mae * 100
    
    print(f"    Raw MAE: {raw_mae:.4f}")
    print(f"    Time-corrected MAE: {corrected_mae_time:.4f} ({improvement_time:+.1f}%)")
    
    # Check per-period improvement
    print(f"\n  Per-Period Results (with correction):")
    for period, (start, end) in [('night', (0, 6)), ('morning', (6, 12)), ('afternoon', (12, 18)), ('evening', (18, 24))]:
        period_mask = (hours >= start) & (hours < end)
        if period_mask.sum() > 10:
            raw_period_mae = np.mean(abs_errors[period_mask])
            corrected_period_mae = np.mean(np.abs(y_test.values[period_mask] - corrected_preds_time[period_mask]))
            pct_change = (raw_period_mae - corrected_period_mae) / raw_period_mae * 100
            print(f"    {period}: {raw_period_mae:.4f} → {corrected_period_mae:.4f} ({pct_change:+.1f}%)")
    
    # Decide whether to use bias correction
    # IMPROVED: Use bias correction if it helps ANY high-error period, even if overall MAE is worse
    high_error_periods = [p for p, tb in bias_factors['by_time'].items() if tb.get('is_high_error', False)]
    helps_high_error = False
    
    for period in high_error_periods:
        (start, end) = {'night': (0, 6), 'morning': (6, 12), 'afternoon': (12, 18), 'evening': (18, 24)}[period]
        period_mask = (hours >= start) & (hours < end)
        if period_mask.sum() > 10:
            raw_period_mae = np.mean(abs_errors[period_mask])
            corrected_period_mae = np.mean(np.abs(y_test.values[period_mask] - corrected_preds_time[period_mask]))
            if corrected_period_mae < raw_period_mae:
                helps_high_error = True
                break
    
    use_bias_correction = corrected_mae_time < raw_mae or helps_high_error
    
    if use_bias_correction:
        print(f"\n  ✓ Bias correction ENABLED")
        if helps_high_error:
            print(f"    Reason: Helps high-error periods: {high_error_periods}")
        bias_corrected_model = create_bias_corrected_predictor(model, scaler, bias_factors, features)
        trained_models[horizon]['bias_corrected_model'] = bias_corrected_model
        trained_models[horizon]['use_bias_correction'] = True
    else:
        print(f"\n  ✗ Bias correction disabled (no improvement)")
        trained_models[horizon]['use_bias_correction'] = False
    
    # Standard error analysis
    analysis = {
        'mean_error': float(np.mean(errors)),
        'std_error': float(np.std(errors)),
        'mae': float(np.mean(abs_errors)),
        'corrected_mae': float(corrected_mae_time),
        'median_ae': float(np.median(abs_errors)),
        'max_error': float(np.max(abs_errors)),
        'p95_error': float(np.percentile(abs_errors, 95)),
        'p99_error': float(np.percentile(abs_errors, 99)),
        'bias_correction_applied': use_bias_correction
    }
    
    # Error by regime
    regime_errors = {}
    for regime_val, regime_name in [(0, 'normal'), (1, 'elevated'), (2, 'spike')]:
        regime_mask = regimes == regime_val
        if regime_mask.sum() > 10:
            regime_errors[regime_name] = float(np.mean(abs_errors[regime_mask]))
    analysis['regime_errors'] = regime_errors
    
    # Error by time
    time_errors = {}
    for period, (start, end) in [('night', (0, 6)), ('morning', (6, 12)), ('afternoon', (12, 18)), ('evening', (18, 24))]:
        period_mask = (hours >= start) & (hours < end)
        if period_mask.sum() > 10:
            time_errors[period] = float(np.mean(abs_errors[period_mask]))
    analysis['time_errors'] = time_errors
    
    error_analysis[horizon] = analysis

print(f"\n{'='*60}")
print("BIAS CORRECTION SUMMARY")
print("="*60)

for horizon, bcf in bias_correction_factors.items():
    use_bc = trained_models.get(horizon, {}).get('use_bias_correction', False)
    status = "ENABLED" if use_bc else "DISABLED"
    print(f"\n{horizon}: {status}")
    
    if use_bc:
        print(f"  Time-specific corrections:")
        for period, tb in bcf['by_time'].items():
            if tb['should_apply']:
                print(f"    {period}: {tb['correction']:+.4f}")

print(f"\n✓ Error analysis complete")

In [ ]:
# AUTOMATED RETRAINING TRIGGERSprint("\n" + "="*60)print("RETRAINING TRIGGER ANALYSIS")print("="*60)retraining_recommendations = {}# Thresholds for retrainingPERFORMANCE_DEGRADATION_THRESHOLD = 0.20  # 20% worse than baselineDISTRIBUTION_SHIFT_THRESHOLD = 1.5  # 1.5 std dev shiftERROR_INCREASE_THRESHOLD = 0.15  # 15% increase in recent errorsdef check_retraining_needed(horizon, metrics, baselines, error_analysis_data):    """Check if model needs retraining based on multiple signals"""    signals = []    should_retrain = False        # 1. Check vs holdout baseline    if 'holdout_best' in baselines.get(horizon.replace('24h', '4h'), {}):        holdout_baseline = baselines[horizon.replace('24h', '4h')]['holdout_best']        model_mae = metrics['mae']                if model_mae > holdout_baseline:            degradation = (model_mae - holdout_baseline) / holdout_baseline            signals.append(f"Model worse than baseline by {degradation*100:.1f}%")            if degradation > PERFORMANCE_DEGRADATION_THRESHOLD:                should_retrain = True        # 2. Check distribution shift    if DISTRIBUTION_SHIFT_DETECTED:        signals.append(f"Distribution shift detected (magnitude: {SHIFT_MAGNITUDE:.2f})")        if SHIFT_MAGNITUDE > DISTRIBUTION_SHIFT_THRESHOLD:            should_retrain = True        # 3. Check error patterns    if horizon in error_analysis:        ea = error_analysis[horizon]                # Check for systematic bias        if abs(ea['mean_error']) > 0.1:            signals.append(f"Systematic bias: {ea['mean_error']:.4f}")            should_retrain = True                # Check P99 errors (catastrophic failures)        if ea['p99_error'] > 3 * ea['mae']:            signals.append(f"High P99 error: {ea['p99_error']:.4f} (3x MAE)")        # 4. Check regime-specific degradation    if horizon in error_analysis and 'regime_errors' in error_analysis[horizon]:        regime_errors = error_analysis[horizon]['regime_errors']        if 'spike' in regime_errors and 'normal' in regime_errors:            spike_ratio = regime_errors['spike'] / (regime_errors['normal'] + 1e-8)            if spike_ratio > 3:                signals.append(f"Spike regime error {spike_ratio:.1f}x worse than normal")        return should_retrain, signalsfor horizon in ['1h', '4h', '24h']:    if horizon not in trained_models:        continue        print(f"\n{horizon} Retraining Analysis:")        metrics = trained_models[horizon]['metrics']    should_retrain, signals = check_retraining_needed(        horizon, metrics, BASELINES,         error_analysis if 'error_analysis' in dir() else {}    )        retraining_recommendations[horizon] = {        'should_retrain': should_retrain,        'signals': signals,        'urgency': 'high' if should_retrain and len(signals) > 2 else 'medium' if should_retrain else 'low'    }        if signals:        for signal in signals:            print(f"  ⚠️ {signal}")    else:        print(f"  ✓ No retraining signals detected")        if should_retrain:        print(f"  >>> RECOMMENDATION: Retrain {horizon} model")    else:        print(f"  >>> Model OK, no retraining needed")# Overall recommendationany_retrain = any(r['should_retrain'] for r in retraining_recommendations.values())high_urgency = any(r['urgency'] == 'high' for r in retraining_recommendations.values())print(f"\n{'='*60}")print("OVERALL RETRAINING RECOMMENDATION")print(f"{'='*60}")if high_urgency:    print("🔴 HIGH URGENCY: Models show significant degradation")    print("   Recommendation: Retrain immediately with fresh data")elif any_retrain:    print("🟡 MEDIUM: Some models could benefit from retraining")    print("   Recommendation: Schedule retraining when convenient")else:    print("🟢 LOW: Models performing within acceptable parameters")    print("   Recommendation: Continue monitoring, retrain in 1-2 weeks")# === NEW: Export retraining trigger file ===retraining_trigger_file = {    'timestamp': datetime.now().isoformat(),    'overall_recommendation': 'retrain_immediately' if high_urgency else ('schedule_retrain' if any_retrain else 'no_action'),    'urgency': 'high' if high_urgency else ('medium' if any_retrain else 'low'),    'horizons': {}}for horizon, rec in retraining_recommendations.items():    retraining_trigger_file['horizons'][horizon] = {        'should_retrain': rec['should_retrain'],        'urgency': rec['urgency'],        'signals': rec['signals']    }# Save to output folder (will be included in zip)import jsontrigger_path = '/content/retraining_needed.json'try:    with open(trigger_path, 'w') as f:        json.dump(retraining_trigger_file, f, indent=2)    print(f"\n✓ Retraining trigger file saved to: {trigger_path}")    print(f"  Recommendation: {retraining_trigger_file['overall_recommendation']}")except Exception as e:    print(f"  ⚠️ Could not save trigger file: {e}")

In [ ]:
# BACKTESTING FRAMEWORK + ONLINE LEARNING HOOKS
print("\n" + "="*60)
print("BACKTESTING + ONLINE LEARNING")
print("="*60)

def run_backtest(model, scaler, features, df, target_col, conformal_interval=None, 
                 time_intervals=None, window_days=3):
    """
    Run walk-forward backtest with interval coverage verification.
    """
    rph = 120
    window_size = window_days * 24 * rph
    
    results = []
    
    for i in range(window_size, len(df) - rph, rph):  # Step by 1 hour
        X_current = df[features].iloc[i:i+1]
        
        if i + rph < len(df):
            y_actual = df[target_col].iloc[i + rph] if target_col in df.columns else np.nan
        else:
            y_actual = np.nan
        
        if pd.isna(y_actual) or X_current.isna().any().any():
            continue
        
        # Predict
        X_scaled = scaler.transform(X_current)
        y_pred = model.predict(X_scaled)[0]
        
        error = abs(y_actual - y_pred)
        
        # Get hour for time-specific interval
        hour = df.index[i].hour if hasattr(df.index[i], 'hour') else 12
        period = 'afternoon' if 12 <= hour < 18 else (
            'morning' if 6 <= hour < 12 else (
            'evening' if 18 <= hour < 24 else 'night'))
        
        # Compute interval coverage
        if conformal_interval:
            base_interval = conformal_interval
            if time_intervals and period in time_intervals:
                interval = time_intervals[period].get('interval_80', base_interval)
            else:
                interval = base_interval
            in_interval = error <= interval
        else:
            in_interval = None
            interval = None
        
        results.append({
            'timestamp': df.index[i],
            'hour': hour,
            'period': period,
            'actual': y_actual,
            'predicted': y_pred,
            'error': error,
            'signed_error': y_actual - y_pred,
            'current_gas': df['gas'].iloc[i] if 'gas' in df.columns else np.nan,
            'in_interval': in_interval,
            'interval_width': interval
        })
    
    return pd.DataFrame(results)

def compute_online_bias_update(recent_errors, decay=0.95, max_history=50):
    """
    Compute exponentially weighted bias update from recent errors.
    Can be used to update bias correction factors online.
    """
    if len(recent_errors) == 0:
        return {'bias': 0.0, 'confidence': 0.0}
    
    n = min(len(recent_errors), max_history)
    errors = np.array(recent_errors[-n:])
    
    # Exponential weights
    weights = np.array([decay ** (n - i - 1) for i in range(n)])
    weights = weights / weights.sum()
    
    weighted_bias = np.sum(errors * weights)
    weighted_abs_error = np.sum(np.abs(errors) * weights)
    
    # Confidence based on consistency
    error_std = np.std(errors)
    confidence = 1.0 / (1.0 + error_std)
    
    return {
        'bias': float(weighted_bias),
        'mae': float(weighted_abs_error),
        'confidence': float(confidence),
        'n_samples': n
    }

class OnlineBiasTracker:
    """Track and update bias corrections online."""
    
    def __init__(self, initial_bias_factors=None, decay=0.95, max_history=100):
        self.decay = decay
        self.max_history = max_history
        self.errors_by_period = {
            'night': [], 'morning': [], 'afternoon': [], 'evening': []
        }
        self.errors_overall = []
        self.bias_factors = initial_bias_factors or {}
    
    def update(self, signed_error, hour):
        """Update with new prediction error."""
        period = 'afternoon' if 12 <= hour < 18 else (
            'morning' if 6 <= hour < 12 else (
            'evening' if 18 <= hour < 24 else 'night'))
        
        self.errors_overall.append(signed_error)
        self.errors_by_period[period].append(signed_error)
        
        # Trim history
        if len(self.errors_overall) > self.max_history:
            self.errors_overall.pop(0)
        if len(self.errors_by_period[period]) > self.max_history // 4:
            self.errors_by_period[period].pop(0)
    
    def get_current_bias(self, period=None):
        """Get current estimated bias for a period or overall."""
        if period and period in self.errors_by_period:
            return compute_online_bias_update(self.errors_by_period[period], self.decay)
        return compute_online_bias_update(self.errors_overall, self.decay)
    
    def should_alert(self, threshold=0.1):
        """Check if any period has significant bias drift."""
        alerts = []
        for period, errors in self.errors_by_period.items():
            if len(errors) >= 10:
                bias_info = compute_online_bias_update(errors, self.decay)
                if abs(bias_info['bias']) > threshold and bias_info['confidence'] > 0.5:
                    alerts.append({
                        'period': period,
                        'bias': bias_info['bias'],
                        'confidence': bias_info['confidence']
                    })
        return alerts

# Store online trackers for each model
online_trackers = {}

backtest_results = {}

for horizon in ['1h', '4h']:
    if horizon not in trained_models:
        continue
    
    print(f"\n{'='*50}")
    print(f"{horizon} Backtest + Online Learning Simulation")
    print(f"{'='*50}")
    
    data = trained_models[horizon]
    model = data['model']
    scaler = data['scaler']
    features = data['features']
    
    target_col = f'target_{horizon}'
    
    if target_col not in df_clean.columns:
        print(f"  ⚠️ Target column not found")
        continue
    
    # Get calibration data
    conformal_interval = conformal_residuals.get(horizon, {}).get('quantile')
    time_intervals = time_period_calibration.get(horizon, {}).get('time', {})
    
    # Run backtest
    bt_results = run_backtest(
        model, scaler, features, df_clean, target_col, 
        conformal_interval, time_intervals, window_days=3
    )
    
    if len(bt_results) < 100:
        print(f"  ⚠️ Insufficient backtest data ({len(bt_results)} points)")
        continue
    
    # === BASIC METRICS ===
    mae = bt_results['error'].mean()
    rmse = np.sqrt((bt_results['error'] ** 2).mean())
    naive_errors = abs(bt_results['actual'] - bt_results['current_gas'])
    naive_mae = naive_errors.mean()
    improvement = (naive_mae - mae) / naive_mae * 100
    
    print(f"\n  Basic Metrics:")
    print(f"    Backtest samples: {len(bt_results)}")
    print(f"    Model MAE: {mae:.4f}, Naive MAE: {naive_mae:.4f}")
    print(f"    Improvement vs naive: {improvement:+.1f}%")
    
    # === INTERVAL COVERAGE ===
    if 'in_interval' in bt_results.columns and bt_results['in_interval'].notna().any():
        overall_coverage = bt_results['in_interval'].mean()
        print(f"\n  Interval Coverage:")
        print(f"    Overall: {overall_coverage:.1%} (target: 80%)")
        
        for period in ['night', 'morning', 'afternoon', 'evening']:
            period_mask = bt_results['period'] == period
            if period_mask.sum() >= 20:
                period_coverage = bt_results.loc[period_mask, 'in_interval'].mean()
                status = "✓" if period_coverage >= 0.75 else "⚠️"
                print(f"    {period}: {period_coverage:.1%} {status}")
    
    # === ROLLING PERFORMANCE ===
    bt_results['date'] = bt_results['timestamp'].dt.date
    daily_mae = bt_results.groupby('date')['error'].mean()
    
    print(f"\n  Rolling Performance:")
    print(f"    Daily MAE range: {daily_mae.min():.4f} - {daily_mae.max():.4f}")
    
    # Performance trend
    if len(daily_mae) >= 3:
        first_half = daily_mae.iloc[:len(daily_mae)//2].mean()
        second_half = daily_mae.iloc[len(daily_mae)//2:].mean()
        trend = (second_half - first_half) / first_half * 100
        trend_str = "improving ↓" if trend < -5 else ("degrading ↑" if trend > 5 else "stable →")
        print(f"    Trend: {trend:+.1f}% ({trend_str})")
    
    # === SIMULATE ONLINE LEARNING ===
    print(f"\n  Online Learning Simulation:")
    tracker = OnlineBiasTracker(
        initial_bias_factors=bias_correction_factors.get(horizon),
        decay=0.95
    )
    
    # Simulate updates
    for _, row in bt_results.iterrows():
        tracker.update(row['signed_error'], row['hour'])
    
    # Check for bias drift alerts
    alerts = tracker.should_alert(threshold=0.05)
    if alerts:
        print(f"    ⚠️ Bias drift detected:")
        for alert in alerts:
            print(f"      {alert['period']}: bias={alert['bias']:+.4f} (conf={alert['confidence']:.2f})")
    else:
        print(f"    ✓ No significant bias drift")
    
    # Show current online bias estimates
    print(f"\n  Online Bias Estimates:")
    for period in ['night', 'morning', 'afternoon', 'evening']:
        bias_info = tracker.get_current_bias(period)
        if bias_info['n_samples'] >= 5:
            print(f"    {period}: {bias_info['bias']:+.4f} (n={bias_info['n_samples']})")
    
    online_trackers[horizon] = tracker
    
    # Store results
    backtest_results[horizon] = {
        'n_samples': len(bt_results),
        'mae': float(mae),
        'rmse': float(rmse),
        'naive_mae': float(naive_mae),
        'improvement_pct': float(improvement),
        'interval_coverage': float(bt_results['in_interval'].mean()) if 'in_interval' in bt_results.columns else None,
        'daily_mae_min': float(daily_mae.min()),
        'daily_mae_max': float(daily_mae.max()),
        'online_alerts': alerts
    }
    
    # Store tracker for saving
    trained_models[horizon]['online_tracker'] = tracker

print(f"\n{'='*60}")
print("BACKTEST SUMMARY")
print("="*60)

for horizon, results in backtest_results.items():
    print(f"\n{horizon}:")
    print(f"  MAE: {results['mae']:.4f} ({results['improvement_pct']:+.1f}% vs naive)")
    if results.get('interval_coverage'):
        print(f"  Coverage: {results['interval_coverage']:.1%}")
    if results.get('online_alerts'):
        print(f"  ⚠️ Alerts: {len(results['online_alerts'])} periods with bias drift")

print(f"\n✓ Backtesting and online learning simulation complete")

In [ ]:
# SHAP EXPLANATIONS FOR MODEL INTERPRETABILITY
print("\n" + "="*60)
print("SHAP EXPLANATIONS")
print("="*60)

# Check if SHAP is available
try:
    import shap
    HAS_SHAP = True
    print("✓ SHAP library available")
except ImportError:
    HAS_SHAP = False
    print("⚠️ SHAP not available. Install with: pip install shap")
    print("   Skipping SHAP analysis")

shap_explainers = {}

if HAS_SHAP:
    for horizon in ['1h', '4h']:
        if horizon not in trained_models:
            continue
        
        print(f"\n{horizon} SHAP Analysis...")
        
        data = trained_models[horizon]
        model = data['model']
        scaler = data['scaler']
        features = data['features']
        
        # Get sample of training data for background
        X_sample = df_train_val[features].sample(min(500, len(df_train_val)), random_state=42)
        X_sample_scaled = scaler.transform(X_sample)
        
        try:
            # Create SHAP explainer based on model type
            model_name = data['metrics']['name']
            
            if 'RF' in model_name or 'GBM' in model_name:
                # Tree-based model
                explainer = shap.TreeExplainer(model)
                shap_values = explainer.shap_values(X_sample_scaled[:100])
            else:
                # Linear or other model - use KernelExplainer
                explainer = shap.KernelExplainer(model.predict, X_sample_scaled[:50])
                shap_values = explainer.shap_values(X_sample_scaled[:50])
            
            # Calculate mean absolute SHAP values
            if isinstance(shap_values, list):
                shap_values = shap_values[0]
            
            mean_shap = np.abs(shap_values).mean(axis=0)
            
            # Create feature importance from SHAP
            shap_importance = dict(zip(features, mean_shap))
            sorted_importance = sorted(shap_importance.items(), key=lambda x: x[1], reverse=True)
            
            print(f"  Top 5 features by SHAP importance:")
            for feat, imp in sorted_importance[:5]:
                print(f"    {feat}: {imp:.4f}")
            
            shap_explainers[horizon] = {
                'explainer': explainer,
                'background_data': X_sample_scaled[:50],
                'feature_names': features,
                'shap_importance': dict(sorted_importance)
            }
            
            # Store in trained_models for later use
            trained_models[horizon]['shap_explainer'] = shap_explainers[horizon]
            
            print(f"  ✓ SHAP explainer created")
            
        except Exception as e:
            print(f"  ⚠️ SHAP analysis failed: {e}")

def explain_prediction(horizon, X_single, feature_names):
    """Generate explanation for a single prediction"""
    if not HAS_SHAP or horizon not in shap_explainers:
        return None
    
    explainer_data = shap_explainers[horizon]
    explainer = explainer_data['explainer']
    
    try:
        shap_values = explainer.shap_values(X_single.reshape(1, -1))
        if isinstance(shap_values, list):
            shap_values = shap_values[0]
        
        # Create explanation
        contributions = list(zip(feature_names, shap_values[0]))
        contributions = sorted(contributions, key=lambda x: abs(x[1]), reverse=True)
        
        explanation = []
        for feat, contrib in contributions[:5]:
            direction = "↑" if contrib > 0 else "↓"
            explanation.append(f"{feat}: {direction}{abs(contrib):.3f}")
        
        return explanation
    except:
        return None

if HAS_SHAP and shap_explainers:
    # Example explanation
    print(f"\nExample prediction explanation:")
    for horizon in shap_explainers:
        sample_X = df_train_val[trained_models[horizon]['features']].iloc[-1:].values
        sample_X_scaled = trained_models[horizon]['scaler'].transform(sample_X)
        
        explanation = explain_prediction(horizon, sample_X_scaled[0], trained_models[horizon]['features'])
        if explanation:
            pred = trained_models[horizon]['model'].predict(sample_X_scaled)[0]
            print(f"  {horizon} prediction = {pred:.4f}")
            print(f"  Top contributors: {', '.join(explanation[:3])}")

print(f"\n✓ SHAP explainers created for: {list(shap_explainers.keys())}")


In [ ]:
# DQN AGENT TRAINING (OPTIONAL)
# This trains a reinforcement learning agent for transaction timing
# Skip if you just need prediction models

TRAIN_DQN = False  # Set to True to train DQN agent

if not TRAIN_DQN:
    print("="*60)
    print("DQN TRAINING SKIPPED (set TRAIN_DQN = True to enable)")
    print("="*60)
    DQN_TRAINED = False

In [ ]:
# DQN Training Implementation (runs only if TRAIN_DQN = True)

if TRAIN_DQN:
    print("\n" + "="*60)
    print("TRAINING DQN AGENT")
    print("="*60)
    
    try:
        import torch
        import torch.nn as nn
        import torch.optim as optim
        from collections import deque
        import random
        
        class DQNNetwork(nn.Module):
            def __init__(self, state_dim, action_dim):
                super().__init__()
                self.net = nn.Sequential(
                    nn.Linear(state_dim, 64),
                    nn.ReLU(),
                    nn.Linear(64, 32),
                    nn.ReLU(),
                    nn.Linear(32, action_dim)
                )
            
            def forward(self, x):
                return self.net(x)
        
        class DQNAgent:
            def __init__(self, state_dim, action_dim):
                self.state_dim = state_dim
                self.action_dim = action_dim
                self.epsilon = 1.0
                self.epsilon_min = 0.05
                self.epsilon_decay = 0.995
                self.gamma = 0.99
                self.lr = 0.001
                self.memory = deque(maxlen=10000)
                self.batch_size = 32
                self.training_steps = 0
                
                self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
                self.model = DQNNetwork(state_dim, action_dim).to(self.device)
                self.target_model = DQNNetwork(state_dim, action_dim).to(self.device)
                self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
                self.update_target()
            
            def update_target(self):
                self.target_model.load_state_dict(self.model.state_dict())
            
            def act(self, state):
                if random.random() < self.epsilon:
                    return random.randint(0, self.action_dim - 1)
                state_t = torch.FloatTensor(state).unsqueeze(0).to(self.device)
                with torch.no_grad():
                    q_values = self.model(state_t)
                return q_values.argmax().item()
            
            def remember(self, state, action, reward, next_state, done):
                self.memory.append((state, action, reward, next_state, done))
            
            def replay(self):
                if len(self.memory) < self.batch_size:
                    return
                
                batch = random.sample(self.memory, self.batch_size)
                states, actions, rewards, next_states, dones = zip(*batch)
                
                states = torch.FloatTensor(states).to(self.device)
                actions = torch.LongTensor(actions).to(self.device)
                rewards = torch.FloatTensor(rewards).to(self.device)
                next_states = torch.FloatTensor(next_states).to(self.device)
                dones = torch.FloatTensor(dones).to(self.device)
                
                current_q = self.model(states).gather(1, actions.unsqueeze(1))
                next_q = self.target_model(next_states).max(1)[0].detach()
                target_q = rewards + (1 - dones) * self.gamma * next_q
                
                loss = nn.MSELoss()(current_q.squeeze(), target_q)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                
                self.training_steps += 1
                if self.training_steps % 100 == 0:
                    self.update_target()
                
                self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)
            
            def save(self, path):
                torch.save(self.model.state_dict(), path)
        
        # Create simple environment
        state_dim = min(30, len(X.columns))  # Limit state size
        action_dim = 2  # 0 = wait, 1 = execute
        
        DQN_AGENT = DQNAgent(state_dim, action_dim)
        
        # Train for a few episodes
        n_episodes = 500
        print(f"Training DQN for {n_episodes} episodes...")
        
        for episode in range(n_episodes):
            # Simple training loop
            for i in range(min(100, len(X) - 1)):
                state = X.iloc[i, :state_dim].values
                action = DQN_AGENT.act(state)
                
                # Simple reward: negative gas price change if executing
                next_gas = current_gas.iloc[i + 1] if i + 1 < len(current_gas) else current_gas.iloc[i]
                reward = -(next_gas - current_gas.iloc[i]) if action == 1 else -0.001  # Small wait penalty
                
                next_state = X.iloc[i + 1, :state_dim].values if i + 1 < len(X) else state
                done = (i >= min(99, len(X) - 2))
                
                DQN_AGENT.remember(state, action, reward, next_state, done)
                DQN_AGENT.replay()
            
            if (episode + 1) % 100 == 0:
                print(f"  Episode {episode + 1}/{n_episodes}, Epsilon: {DQN_AGENT.epsilon:.3f}")
        
        DQN_TRAINED = True
        DQN_METRICS = {
            'episodes': n_episodes,
            'training_steps': DQN_AGENT.training_steps,
            'final_epsilon': float(DQN_AGENT.epsilon)
        }
        print(f"\n✓ DQN training complete ({DQN_AGENT.training_steps} steps)")
        
    except ImportError:
        print("⚠️ PyTorch not available, skipping DQN training")
        DQN_TRAINED = False
    except Exception as e:
        print(f"⚠️ DQN training failed: {e}")
        DQN_TRAINED = False
else:
    DQN_TRAINED = False

In [ ]:
# Save all models + ALL IMPROVEMENTS
import os
from datetime import datetime
import json as json_lib

os.makedirs('saved_models', exist_ok=True)

print("\n" + "="*60)
print("SAVING MODELS AND ANALYSIS")
print("="*60)

# === Save prediction models ===
for horizon in ['1h', '4h', '24h']:
    if horizon not in trained_models:
        continue
    
    data = trained_models[horizon]
    model = data['model']
    scaler = data['scaler']
    metrics = data['metrics']
    features = data.get('features', [])
    
    model_data = {
        'model': model,
        'model_name': metrics['name'],
        'metrics': {
            'mae': float(metrics['mae']),
            'improvement': float(metrics['improvement']),
            'vs_holdout_baseline': float(metrics['vs_holdout_baseline']) if metrics.get('vs_holdout_baseline') else None,
            'passed_baseline': bool(metrics.get('passed_baseline', False)),
            'is_ensemble': bool(metrics.get('is_ensemble', False)),
        },
        'trained_at': datetime.now().isoformat(),
        'feature_names': list(features),
        'feature_scaler': scaler,
    }
    
    if metrics.get('is_ensemble'):
        model_data['ensemble_weights'] = metrics.get('ensemble_weights')
    
    # Add bias correction if available
    if data.get('use_bias_correction') and horizon in bias_correction_factors:
        model_data['bias_correction'] = bias_correction_factors[horizon]
        model_data['use_bias_correction'] = True
    
    # Add bias-corrected model wrapper
    if 'bias_corrected_model' in data:
        model_data['bias_corrected_model'] = data['bias_corrected_model']
    
    # Add time-adaptive predictor
    if 'time_adaptive_predict' in data:
        model_data['has_time_adaptive'] = True
    
    # Add confidence scoring info
    if data.get('has_confidence_scoring'):
        model_data['has_confidence_scoring'] = True
    
    # Add online tracker initial state
    if 'online_tracker' in data:
        tracker = data['online_tracker']
        model_data['online_bias_state'] = {
            'overall': tracker.get_current_bias(),
            'by_period': {p: tracker.get_current_bias(p) for p in ['night', 'morning', 'afternoon', 'evening']}
        }
    
    if 'conformal_residuals' in dir() and horizon in conformal_residuals:
        model_data['conformal_interval'] = float(conformal_residuals[horizon]['quantile'])
        model_data['conformal_calibration_source'] = conformal_residuals[horizon].get('calibration_source', 'training')
    
    if 'time_period_calibration' in dir() and horizon in time_period_calibration:
        model_data['time_calibration'] = time_period_calibration[horizon]
    
    joblib.dump(model_data, f'saved_models/model_{horizon}.pkl')
    print(f"✓ model_{horizon}.pkl ({metrics['name']}, {len(features)} features)")
    joblib.dump(scaler, f'saved_models/scaler_{horizon}.pkl')

# === Save time-specific models ===
if 'time_specific_models' in dir() and time_specific_models:
    os.makedirs('saved_models/time_models', exist_ok=True)
    for horizon, periods in time_specific_models.items():
        for period, pdata in periods.items():
            filename = f'saved_models/time_models/model_{horizon}_{period}.pkl'
            joblib.dump(pdata, filename)
            print(f"  → {horizon}_{period} time model")

# === Save regime-specific models ===
if 'regime_specific_models' in dir() and regime_specific_models:
    os.makedirs('saved_models/regime_models', exist_ok=True)
    for horizon, regime_dict in regime_specific_models.items():
        for regime_val, regime_data in regime_dict.items():
            filename = f'saved_models/regime_models/model_{horizon}_{regime_data["regime_name"]}.pkl'
            joblib.dump(regime_data, filename)
            print(f"  → {horizon}_{regime_data['regime_name']} regime")

# === Save spike forecast models ===
if 'spike_forecast_models' in dir() and spike_forecast_models:
    for horizon, sf_data in spike_forecast_models.items():
        sf_save = {
            'model': sf_data.get('model'),
            'scaler': sf_data.get('scaler'),
            'features': sf_data.get('features', []),
            'threshold_config': sf_data.get('threshold_config'),
            'optimal_threshold': sf_data.get('optimal_threshold', 0.5),
            'metrics': {k: v for k, v in sf_data.items() if k not in ['model', 'scaler', 'features', 'threshold_config']}
        }
        joblib.dump(sf_save, f'saved_models/spike_forecast_{horizon}.pkl')
        if sf_data.get('model') is not None:
            print(f"✓ spike_forecast_{horizon}.pkl (AUC: {sf_data.get('auc', 0.5):.3f})")
        else:
            print(f"⚠️ spike_forecast_{horizon}.pkl (no model)")

# === Save other files ===
default_features = trained_models.get('4h', trained_models.get('1h', {})).get('features', [])
joblib.dump(list(default_features), 'saved_models/feature_names.pkl')

if 'regime_clf' in dir() and regime_clf is not None:
    joblib.dump({'model': regime_clf, 'scaler': regime_scaler, 'accuracy': regime_accuracy}, 
                'saved_models/regime_detector.pkl')

if 'quantile_models' in dir() and quantile_models:
    for horizon, (q_models, q_scaler) in quantile_models.items():
        quantile_data = {'models': q_models, 'scaler': q_scaler, 'quantiles': [0.1, 0.5, 0.9]}
        if 'conformal_residuals' in dir() and horizon in conformal_residuals:
            quantile_data['conformal'] = {
                'interval_width': float(conformal_residuals[horizon]['quantile']),
                'calibration_source': conformal_residuals[horizon].get('calibration_source', 'training')
            }
        if 'time_period_calibration' in dir() and horizon in time_period_calibration:
            quantile_data['time_calibration'] = time_period_calibration[horizon]
        joblib.dump(quantile_data, f'saved_models/quantile_{horizon}.pkl')

# === Save training metadata ===
def convert_to_python_types(obj):
    if isinstance(obj, dict):
        return {k: convert_to_python_types(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_python_types(v) for v in obj]
    elif isinstance(obj, (np.bool_, np.integer)):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif hasattr(obj, 'item'):
        return obj.item()
    else:
        return obj

metadata = {
    'training_timestamp': datetime.now().isoformat(),
    'total_samples': len(df_clean),
    'training_samples': len(df_train_val),
    'holdout_samples': len(df_holdout) if df_holdout is not None else 0,
    'date_range': f"{df_clean.index.min()} to {df_clean.index.max()}",
    'selection_method': 'holdout-based',
    'configuration': {
        'target_transform': TARGET_TRANSFORM_USED if 'TARGET_TRANSFORM_USED' in dir() else 'none',
        'use_rolling_window': USE_ROLLING_WINDOW if 'USE_ROLLING_WINDOW' in dir() else False,
        'rolling_window_days': ROLLING_WINDOW_DAYS if 'ROLLING_WINDOW_DAYS' in dir() else None,
        'distribution_shift_detected': DISTRIBUTION_SHIFT_DETECTED if 'DISTRIBUTION_SHIFT_DETECTED' in dir() else False,
        'shift_magnitude': SHIFT_MAGNITUDE if 'SHIFT_MAGNITUDE' in dir() else 0,
        'feature_pruning_enabled': ENABLE_FEATURE_PRUNING if 'ENABLE_FEATURE_PRUNING' in dir() else False,
        'ensemble_blending_enabled': USE_ENSEMBLE_BLENDING if 'USE_ENSEMBLE_BLENDING' in dir() else False,
        'regularization_strength': REGULARIZATION_STRENGTH if 'REGULARIZATION_STRENGTH' in dir() else 0,
        'time_specific_models_enabled': TRAIN_TIME_SPECIFIC_MODELS if 'TRAIN_TIME_SPECIFIC_MODELS' in dir() else False,
    },
    'features': {'count': len(default_features), 'list': list(default_features)},
    'baselines': BASELINES,
    'models': {},
    'time_specific_models': {},
    'regime_models': {},
    'direction_models': {},
    'spike_forecast_models': {},
    'error_analysis': {},
    'bias_correction': {},
    'calibration': {},
    'retraining_recommendations': {},
    'backtest_results': {}
}

# Model info
for horizon, data in trained_models.items():
    m = data['metrics']
    metadata['models'][horizon] = {
        'name': m['name'],
        'mae': float(m['mae']),
        'improvement_pct': float(m['improvement'] * 100),
        'vs_holdout_baseline_pct': float(m['vs_holdout_baseline'] * 100) if m.get('vs_holdout_baseline') else None,
        'passed_baseline': bool(m.get('passed_baseline', False)),
        'is_fallback': data.get('is_fallback', False),
        'is_ensemble': m.get('is_ensemble', False),
        'has_bias_correction': data.get('use_bias_correction', False),
        'has_time_adaptive': 'time_adaptive_predict' in data,
        'n_features': len(data.get('features', []))
    }
    if 'calibration' in data:
        metadata['models'][horizon]['calibration'] = data['calibration']

# Time-specific models
if 'time_specific_models' in dir() and time_specific_models:
    for horizon, periods in time_specific_models.items():
        metadata['time_specific_models'][horizon] = {
            period: {'mae': float(p['mae']), 'improvement': float(p['improvement'])}
            for period, p in periods.items()
        }

# Regime models
if 'regime_specific_models' in dir() and regime_specific_models:
    for horizon, regime_dict in regime_specific_models.items():
        metadata['regime_models'][horizon] = {}
        for regime_val, regime_data in regime_dict.items():
            metadata['regime_models'][horizon][regime_data['regime_name']] = {
                'mae': float(regime_data['metrics']['mae']),
                'n_samples': int(regime_data['n_samples'])
            }

# Direction models
if 'direction_models' in dir() and direction_models:
    for horizon, data in direction_models.items():
        metadata['direction_models'][horizon] = {
            'accuracy': float(data['accuracy']),
            'f1_score': float(data['f1_score']),
            'baseline_accuracy': float(data.get('baseline_accuracy', 0)),
            'improvement_vs_baseline': float(data.get('improvement_vs_baseline', 0)),
        }

# Spike forecast
if 'spike_forecast_models' in dir() and spike_forecast_models:
    for horizon, sf_data in spike_forecast_models.items():
        metadata['spike_forecast_models'][horizon] = {
            'has_model': sf_data.get('model') is not None,
            'precision': float(sf_data.get('precision', 0)),
            'recall': float(sf_data.get('recall', 0)),
            'f1_score': float(sf_data.get('f1_score', 0)),
            'auc': float(sf_data.get('auc', 0.5)),
            'threshold_config': sf_data.get('threshold_config', {}).get('name') if sf_data.get('threshold_config') else None,
        }

# Error analysis
if 'error_analysis' in dir() and error_analysis:
    metadata['error_analysis'] = error_analysis

# Bias correction
if 'bias_correction_factors' in dir() and bias_correction_factors:
    metadata['bias_correction'] = bias_correction_factors

# Calibration info
if 'conformal_residuals' in dir():
    for horizon in conformal_residuals:
        if horizon not in metadata['calibration']:
            metadata['calibration'][horizon] = {}
        metadata['calibration'][horizon]['conformal_width'] = float(conformal_residuals[horizon]['quantile'])
        metadata['calibration'][horizon]['source'] = conformal_residuals[horizon].get('calibration_source', 'training')

# Retraining recommendations
if 'retraining_recommendations' in dir() and retraining_recommendations:
    metadata['retraining_recommendations'] = retraining_recommendations

# Backtest results
if 'backtest_results' in dir() and backtest_results:
    metadata['backtest_results'] = {h: {k: v for k, v in r.items() if k != 'online_alerts'} 
                                    for h, r in backtest_results.items()}

metadata = convert_to_python_types(metadata)

with open('saved_models/training_metadata.json', 'w') as f:
    json_lib.dump(metadata, f, indent=2)
print(f"\n✓ training_metadata.json")

# Feature importance
if 'FEATURE_IMPORTANCE' in dir() and FEATURE_IMPORTANCE:
    sorted_importance = dict(sorted(FEATURE_IMPORTANCE.items(), key=lambda x: x[1], reverse=True))
    with open('saved_models/feature_importance.json', 'w') as f:
        json_lib.dump(convert_to_python_types(sorted_importance), f, indent=2)
    print(f"✓ feature_importance.json")

# Training history
history_file = 'saved_models/training_history.json'
if os.path.exists(history_file):
    with open(history_file) as f:
        history = json_lib.load(f)
else:
    history = []

current_run = {
    'timestamp': datetime.now().isoformat(),
    'models': {h: {
        'name': trained_models[h]['metrics']['name'], 
        'mae': float(trained_models[h]['metrics']['mae']),
        'is_ensemble': trained_models[h]['metrics'].get('is_ensemble', False),
        'has_bias_correction': trained_models[h].get('use_bias_correction', False)
    } for h in trained_models},
    'improvements': {
        'ensemble_blending': USE_ENSEMBLE_BLENDING if 'USE_ENSEMBLE_BLENDING' in dir() else False,
        'time_specific_models': bool(time_specific_models) if 'time_specific_models' in dir() else False,
        'bias_correction': any(trained_models.get(h, {}).get('use_bias_correction', False) for h in ['1h', '4h']),
        'holdout_calibration': True
    }
}

history.append(current_run)
history = history[-10:]

with open(history_file, 'w') as f:
    json_lib.dump(history, f, indent=2)

print(f"\n{'='*60}")
print("SUMMARY")
print("="*60)

for horizon, data in trained_models.items():
    m = data['metrics']
    extras = []
    if m.get('is_ensemble'):
        extras.append("ensemble")
    if data.get('use_bias_correction'):
        extras.append("bias-corrected")
    if 'time_adaptive_predict' in data:
        extras.append("time-adaptive")
    
    extras_str = f" [{', '.join(extras)}]" if extras else ""
    print(f"✓ {horizon}: {m['name']} | MAE: {m['mae']:.4f}{extras_str}")

print(f"\n{'='*60}")
print("ALL MODELS SAVED")
print("="*60)

In [ ]:
# Print final report
print("\n" + "="*70)
print("TRAINING COMPLETE - FINAL REPORT")
print("="*70)

total_days = len(df_clean) / (120 * 24)

print(f"\nDATA SUMMARY")
print(f"   Total samples: {len(df_clean):,} ({total_days:.1f} days)")
print(f"   Training: {len(df_train_val):,} | Holdout: {len(df_holdout) if df_holdout is not None else 0:,}")
print(f"   Date range: {df_clean.index.min()} to {df_clean.index.max()}")
print(f"   ETH price: {'Binance 1-min ✓' if HAS_ETH_PRICE else 'Not available'}")
print(f"   Features: 1h={len(numeric_features_1h)}, 4h={len(numeric_features_4h)}, 24h={len(numeric_features_24h)}")

print(f"\n" + "-"*70)
print(f"{'MODEL PERFORMANCE':^70}")
print("-"*70)
print(f"{'Horizon':<8} {'Model':<15} {'CV MAE':>10} {'Holdout':>10} {'vs Base':>10} {'Status':>12}")
print("-"*70)

for horizon in ['1h', '4h', '24h']:
    if horizon in trained_models:
        data = trained_models[horizon]
        m = data['metrics']
        name = m['name'][:14]
        if data.get('is_fallback'):
            name = name[:10] + '(fb)'
        
        cv_mae = f"{m['mae']:.4f}"
        holdout_mae = f"{data.get('holdout_mae', 0):.4f}" if 'holdout_mae' in data else "N/A"
        improvement = f"{m['improvement']*100:+.1f}%"
        status = "✓ PASS" if m.get('passed_baseline', False) else "✗ FAIL"
        
        print(f"{horizon:<8} {name:<15} {cv_mae:>10} {holdout_mae:>10} {improvement:>10} {status:>12}")

print("-"*70)

# Calibration report
if any('calibration' in trained_models.get(h, {}) for h in ['1h', '4h']):
    print(f"\n" + "-"*70)
    print(f"{'PREDICTION INTERVAL CALIBRATION':^70}")
    print("-"*70)
    print(f"{'Horizon':<10} {'Conformal 80%':>15} {'Adaptive 80%':>15} {'Width':>15}")
    print("-"*70)
    
    for horizon in ['1h', '4h']:
        if horizon in trained_models and 'calibration' in trained_models[horizon]:
            cal = trained_models[horizon]['calibration']
            c_cov = f"{cal.get('conformal_coverage', 0):.1%}"
            a_cov = f"{cal.get('adaptive_coverage', 0):.1%}"
            width = f"±{cal.get('conformal_width', 0):.4f}"
            print(f"{horizon:<10} {c_cov:>15} {a_cov:>15} {width:>15}")
    
    print("-"*70)

# Direction models
if 'direction_models' in dir() and direction_models:
    print(f"\n" + "-"*70)
    print(f"{'DIRECTION PREDICTION':^70}")
    print("-"*70)
    for horizon, data in direction_models.items():
        print(f"  {horizon}: Accuracy={data['accuracy']:.1%}, F1={data['f1_score']:.3f}")

# Spike forecast
if 'spike_forecast_models' in dir() and spike_forecast_models:
    print(f"\n" + "-"*70)
    print(f"{'SPIKE FORECASTING':^70}")
    print("-"*70)
    for horizon, data in spike_forecast_models.items():
        if data.get('model') is not None:
            print(f"  {horizon}: AUC={data.get('auc', 0.5):.3f}, P={data.get('precision', 0):.1%}, R={data.get('recall', 0):.1%}")
        else:
            print(f"  {horizon}: No viable model (AUC < 0.55)")

# Bias correction
if 'bias_correction_factors' in dir() and bias_correction_factors:
    print(f"\n" + "-"*70)
    print(f"{'BIAS CORRECTION':^70}")
    print("-"*70)
    for horizon, bcf in bias_correction_factors.items():
        use_bc = trained_models.get(horizon, {}).get('use_bias_correction', False)
        if use_bc:
            print(f"  {horizon}: ENABLED (correction: {bcf['overall']['correction']:+.4f})")
        else:
            print(f"  {horizon}: disabled")

# 24h model status
print(f"\n" + "-"*70)
print(f"{'24H MODEL STATUS':^70}")
print("-"*70)
if '24h' in trained_models:
    if trained_models['24h'].get('is_fallback'):
        print(f"  ⚠️ Using 4h model as fallback (need 30+ days of data)")
        print(f"     Current data: {total_days:.1f} days")
        print(f"     Recommendation: Collect {30 - total_days:.0f} more days before training true 24h model")
    else:
        print(f"  ✓ True 24h model trained with {total_days:.1f} days of data")

# Final recommendation
print(f"\n" + "="*70)
print("RECOMMENDATION")
print("="*70)

all_passed = all(trained_models.get(h, {}).get('metrics', {}).get('passed_baseline', False) 
                 for h in trained_models if h in trained_models)

if all_passed:
    print("✓ All models beat baseline - READY FOR DEPLOYMENT")
    print("\nNext steps:")
    print("  1. Download saved_models/ folder")
    print("  2. Copy to backend/models/saved_models/")
    print("  3. Restart backend")
else:
    failed = [h for h in trained_models 
              if not trained_models[h]['metrics'].get('passed_baseline', False)]
    print(f"⚠️ Some models did not pass baseline: {failed}")
    print("\nRecommendations:")
    print("  - Collect more data")
    print("  - Review feature engineering")
    print("  - Consider deploying passing models only")

In [ ]:
# Visualizations - IMPROVED with holdout baseline comparison
import matplotlib.pyplot as plt

print("\n" + "="*60)
print("GENERATING VISUALIZATIONS")
print("="*60)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Train vs Holdout Distribution Comparison
ax1 = axes[0, 0]
train_gas = current_gas.values
if HAS_HOLDOUT:
    holdout_gas = df_holdout['gas'].values
    ax1.hist(train_gas, bins=50, alpha=0.6, color='blue', label=f'Train (mean={train_gas.mean():.2f})', density=True)
    ax1.hist(holdout_gas, bins=50, alpha=0.6, color='red', label=f'Holdout (mean={holdout_gas.mean():.2f})', density=True)
    ax1.legend()
    ax1.set_title('Train vs Holdout Distribution')
else:
    ax1.hist(train_gas, bins=50, alpha=0.7, color='blue', edgecolor='black')
    ax1.axvline(train_gas.mean(), color='red', linestyle='--', label=f'Mean: {train_gas.mean():.2f}')
    ax1.legend()
    ax1.set_title('Gas Price Distribution')
ax1.set_xlabel('Gas Price (gwei)')
ax1.set_ylabel('Density' if HAS_HOLDOUT else 'Frequency')

# 2. Model vs HOLDOUT Baseline (not train baseline!)
ax2 = axes[0, 1]
horizons = list(trained_models.keys())
maes = [trained_models[h]['metrics']['mae'] for h in horizons]

# Use holdout baselines if available, otherwise train baselines
baselines = []
for h in horizons:
    h_key = h.replace('24h', '4h')  # 24h uses 4h baseline
    if 'holdout_best' in BASELINES.get(h_key, {}):
        baselines.append(BASELINES[h_key]['holdout_best'])
    else:
        baselines.append(BASELINES.get(h_key, BASELINES['4h'])['best'])

x = np.arange(len(horizons))
width = 0.35
bars1 = ax2.bar(x - width/2, maes, width, label='Model MAE', color='steelblue')
bars2 = ax2.bar(x + width/2, baselines, width, label='Holdout Baseline', color='coral')
ax2.set_xlabel('Horizon')
ax2.set_ylabel('MAE (gwei)')
ax2.set_title('Model vs Holdout Baseline Performance')
ax2.set_xticks(x)
ax2.set_xticklabels(horizons)
ax2.legend()

# Add improvement percentages (vs holdout baseline)
for i, (h, m, b) in enumerate(zip(horizons, maes, baselines)):
    imp = (b - m) / b * 100
    color = 'green' if imp > 0 else 'red'
    y_pos = max(m, b) + 0.02 * max(max(maes), max(baselines))
    ax2.annotate(f'{imp:+.1f}%', xy=(i, y_pos), ha='center', fontsize=10, fontweight='bold', color=color)

# 3. Gas price time series with regime markers
ax3 = axes[1, 0]
sample_size = min(2000, len(df_clean))
sample_df = df_clean.iloc[-sample_size:]
sample_gas = sample_df['gas']

ax3.plot(sample_gas.index, sample_gas.values, linewidth=0.5, alpha=0.8, color='blue')

# Mark holdout period
if HAS_HOLDOUT:
    holdout_start = df_holdout.index[0]
    ax3.axvline(holdout_start, color='red', linestyle='--', linewidth=2, label='Holdout start')
    ax3.legend()

ax3.set_xlabel('Time')
ax3.set_ylabel('Gas Price (gwei)')
ax3.set_title(f'Recent Gas Prices (last {sample_size} samples)')
ax3.tick_params(axis='x', rotation=45)

# 4. Feature importance (top 10)
ax4 = axes[1, 1]
if FEATURE_IMPORTANCE and any(v != list(FEATURE_IMPORTANCE.values())[0] for v in FEATURE_IMPORTANCE.values()):
    # Non-uniform importance
    sorted_imp = sorted(FEATURE_IMPORTANCE.items(), key=lambda x: x[1], reverse=True)[:10]
    features_plot = [f[0][:20] for f in sorted_imp]
    importances = [f[1] for f in sorted_imp]
    
    y_pos = np.arange(len(features_plot))
    ax4.barh(y_pos, importances, color='teal')
    ax4.set_yticks(y_pos)
    ax4.set_yticklabels(features_plot)
    ax4.invert_yaxis()
    ax4.set_xlabel('Importance')
    ax4.set_title('Top 10 Feature Importance (Permutation)')
else:
    ax4.text(0.5, 0.5, 'Feature importance uniform\n(Huber model)', ha='center', va='center', fontsize=12)
    ax4.set_title('Feature Importance')
    ax4.set_xlim(0, 1)
    ax4.set_ylim(0, 1)

plt.tight_layout()
plt.savefig('saved_models/training_results.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Saved training_results.png")

# === ADDITIONAL: Distribution shift visualization ===
if HAS_HOLDOUT and DISTRIBUTION_SHIFT_DETECTED:
    fig2, axes2 = plt.subplots(1, 2, figsize=(12, 4))
    
    for i, horizon in enumerate(['1h', '4h']):
        ax = axes2[i]
        train_target = df_train_val[f'target_{horizon}'].dropna()
        holdout_target = df_holdout[f'target_{horizon}'].dropna()
        
        ax.hist(train_target, bins=50, alpha=0.6, color='blue', label='Train', density=True)
        ax.hist(holdout_target, bins=50, alpha=0.6, color='red', label='Holdout', density=True)
        ax.set_xlabel(f'{horizon} Target (gwei)')
        ax.set_ylabel('Density')
        ax.set_title(f'{horizon} Target Distribution')
        ax.legend()
    
    plt.tight_layout()
    plt.savefig('saved_models/distribution_shift.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("✓ Saved distribution_shift.png")


In [ ]:
# Create zip file for download
import shutil

shutil.make_archive('gweizy_models', 'zip', 'saved_models')
print("\n✅ Created gweizy_models.zip")
print("\nDownload this file and extract to: backend/models/saved_models/")

# Auto-download
files.download('gweizy_models.zip')